In [5]:
from ipyleaflet import *
from ipywidgets import *
from random import *

import pandas as pd
import numpy as np
import math

import xarray as xr
import os
import scipy

import pyqtree as qt
import bbox as bb

from collections import *
import operator
import time
from datetime import timedelta,datetime


In [958]:
class mappDataFrame():
    
    def __init__(self, dataType = [], dataDate = None, airport = None):
    
        self.dataType = []
        self.dataDate = None
        self.airport = None
        self.adsbDate = None
        self.adsbDateHyphen = None
        self.tfms = None
        self.adsbDf = None
        self.tfmsDf = None
        self.aeroDf = None
        self.uasFlights = None
        self.adiHigh = None
        self.adiLow = None
        self.timeOut = []
        self.adsbFlightsDictionary = {}
        self.adsbFlights = None
        self.adsbLines = None
        self.adsbDisplay = None
        self.adsbHeat = {}
        self.uasOd = None
        self.uasLines = None
        self.uasDisplay = None
        self.uasHeat = {}
        self.heatDistanceDelimiter = None
        self.altitudeDelimiter = None
        self.encounterResults = None
        self.encounterFlights = None
        self.heatCircles = None
        self.heatDisplay = None
        self.mapp = None
        
    def buildDataFrames(self, dataType, dataDate, airport = None):
        
        for dataTypes in dataType:
            self.dataType.append(dataTypes)
        
        if 'ADSB' in self.dataType:
            adsbDate = dataDate
            self.adsbDateHyphen = '-'.join(adsbDate)
            self.adsbDate = ''.join(adsbDate)
            self.adsbDf = pd.read_csv('./ADSB/' + self.adsbDate +'_csv.txt')
            
        
        if 'TFMS' in self.dataType:
            
            self.airport = airport
            self.tfmsDf = pd.read_csv('./TFMS/'+ self.airport + ".csv")
            
        if 'AERO' in self.dataType:
            
            xls = pd.ExcelFile('./AERO/aero8-14.xlsx')
            sheet_to_df_map = {}
            for sheet_name in xls.sheet_names:
                sheet_to_df_map[sheet_name] = xls.parse(sheet_name)
                sheet_to_df_map[sheet_name] = sheet_to_df_map[sheet_name].sort_values(['TimeWhen'])
                
            uasFlights = {}
            for df in sheet_to_df_map:
                
                #datetime_object = datetime.strptime(timeIn, '%Y-%m-%d %H:%M:%S')
                
                #('datetime64[ns]')
                
                testDf = pd.to_datetime(sheet_to_df_map[df]['TimeWhen'],#.astype('datetime64[ns]'),
                                        format = '%Y-%m-%d %H:%M:%S') 
                
                testDf = testDf + pd.DateOffset(hours=6)
                
                
                #+ timedelta(hours=9)).strftime('%H:%M:%S')
                
                #segments_data['time'] = pd.DatetimeIndex(segments_data['time']) + timedelta(hours=5,minutes=30)
                
                
                
                #strftime()
                
                #testDf = testDf.apply(lambda x: (dt.datetime.combine(dt.datetime(1,1,1), x,) + dt.timedelta(hours=6)).time())
                
                #pd.DatetimeIndex(montdist['date']) + pd.DateOffset(1)
                

                uasFlights[str(sheet_to_df_map[df]['Drone ID'][0])] =[ [sheet_to_df_map[df]['Latitude'],
                                                                       sheet_to_df_map[df]['Longitude']],
                                                                      testDf.astype(str),
                                                                      sheet_to_df_map[df]['IconAltitude']]
            self.uasFlights = uasFlights

        
    def rebuildAdsbDf(self):
        
        if 'ADSB' in self.dataType:
            adsbDate = self.dataDate
            self.adsbDateHyphen = '-'.join(adsbDate)
            self.adsbDate = ''.join(adsbDate)
            self.adsbDf = pd.read_csv('./ADSB/' + self.adsbDate +'_csv.txt')

        
    def refreshParameters(self, adiLow, adiHigh, timeOut):
        
        self.adiLow = adiLow
        self.adiHigh = adiHigh
        self.timeOut = timeOut
        
    def refreshAltitude(self, adiLow, adiHigh):
        
        self.adiLow = adiLow
        self.adiHigh = adiHigh

    def refreshTime(self, timeOut):
        self.timeOut = timeOut
                    
    def refreshDate(self, date):
        self.dataDate = date
        
    def RGBtoHex(self, RGB):
        # Components need to be integers for hex to make sense
        RGB = [int(x) for x in RGB]
        return "#"+"".join(["0{0:x}".format(v) if v < 16 else
                "{0:x}".format(v) for v in RGB])


    def buildAdsbFlights(self):
        

        flights = []
        flights.extend({self.adsbDf.iat[0,11]})

        for y in range(len(self.adsbDf)):

            acidCur = self.adsbDf.iat[y - 1 , 11]

            if(self.adsbDf.iat[y, 11] != acidCur ):

                if(acidCur in flights):
                    ;
                else:
                    flights.extend([acidCur])

        for i in flights:

            if pd.isnull(i) == True:
                flights.remove(i)

        self.adsbFlights = flights
        
    def refreshAdsbFlights(self):

        adsbFlightsDictionary = {}
        adiLow = self.adiLow
        adiHigh = self.adiHigh
        timeOut = self.timeOut

        for items in self.adsbFlights:

            if type(items) == str:

                adsbFlightsDictionary.update({items+'track':[]})
                adsbFlightsDictionary.update({items+'time':[]})
                adsbFlightsDictionary.update({items+'altitude':[]})  
                
        for x in range(len(self.adsbDf)):

            timeIn = self.adsbDf.iat[x,1]
            timeIn = time.strftime('%H:%M:%S', time.gmtime(timeIn))
            timeIn = ' '.join([self.adsbDateHyphen, timeIn])
            
            if type(self.adsbDf.iat[x,11]) == str:
                
                
                if timeIn >= self.adsbDateHyphen + timeOut[0] and timeIn <= self.adsbDateHyphen + timeOut[1]:
                    if int(self.adsbDf.iat[x,7]) >= adiLow and int(self.adsbDf.iat[x,7]) <= adiHigh:
                        adsbFlightsDictionary[self.adsbDf.iat[x,11] + "track"].append([self.adsbDf.iat[x,4] , self.adsbDf.iat[x,5]])    
                        adsbFlightsDictionary[self.adsbDf.iat[x,11] + "time"].append(timeIn)
                        adsbFlightsDictionary[self.adsbDf.iat[x,11] + "altitude"].append([self.adsbDf.iat[x,10], self.adsbDf.iat[x,7]])
                        
        self.adsbFlightsDictionary = adsbFlightsDictionary

    def refreshAdsbDisplay(self):
        
        lp = 0
        lines = []
        colorRange = 255/len(self.adsbFlights)
        
        for items in self.adsbFlights:
            if lp == 0:
                colorIn = self.RGBtoHex([0,0,255])
            else:
                colorIn = self.RGBtoHex([ 0, int(255-(colorRange * lp)), 255 ])
            
            flightName = items
            
            items = Polyline(
            locations = 
                [[
                    self.adsbFlightsDictionary[items + 'track']
                ,]],
                color = colorIn,
                fill_color = colorIn,
                fill = False,
                opacity = .7)

            if(type(self.adsbFlights[lp]) == str):
                items.popup=HTML(flightName)

            lines.append(items)
            lp+=1;
            
            
        self.adsbLines = lines
        if self.adsbDisplay == None:
            self.adsbDisplay = LayerGroup(layers=(lines)) 
        else:
            self.adsbDisplay.layers = self.adsbLines
            
            
    def refreshUasFlights(self):
        flightTracks = []
        flightTimes = []
        flightAltitudes = []
        uasOd = {}

        timeMin = self.adsbDateHyphen + self.timeOut[0]

        
        timeMax = self.adsbDateHyphen + self.timeOut[1]
        
        
        adiLow = self.adiLow
        adiHigh = self.adiHigh

        #timeCorrect = timedelta(hours = 6)
        
        for z in self.uasFlights:
            tracks = []
            times = []
            altitudes = []
            heatIterator = 0
            for i in range(len(self.uasFlights[z][0][0])):
                k = self.uasFlights[z][0][0][i]
                j = self.uasFlights[z][0][1][i]
                
                
                timeIn = self.uasFlights[z][1][i]
                
#                 datetime_object = datetime.strptime(timeIn, '%Y-%m-%d %H:%M:%S')
#                 timeCheck = (datetime_object + timedelta(hours=6))
                
                
                
                altitude = self.uasFlights[z][2][i]
                if type(timeIn) == str:
                    
                    
                    #'2019-08-14 09:25:36'
                            
                    #timeIn = time.strptime(timeIn, )
                    
#                     timeIn = time.strptime(timeIn, '%Y-%m-%d %H:%M:%S')

#                     #print(type(timeIn))
#                     #timeIn = time.strptime(timeIn, )
#                     #timeIn = timeIn+timeCorrect
#                     timeMin=str(timeMin)
#                     timeMin = time.strptime(timeMin, '%Y-%m-%d %H:%M:%S')
#                     #print(type(timeMin))
#                     timeMax=str(timeMax)
#                     timeMax = time.strptime(timeMax, '%Y-%m-%d %H:%M:%S')
                    
                    
                    if timeIn >= timeMin and timeIn <= timeMax:
                        timeIn = time.strftime(timeIn)
                        if altitude >= adiLow and altitude <= adiHigh:
                            tracks.append([k,j])
                            times.append(timeIn)
                            altitudes.append(altitude)
                            self.uasHeat.update({z + str(heatIterator):[]})
                            heatIterator+=1
    

            flightTimes.append(times)
            flightTracks.append(tracks)
            flightAltitudes.append(altitudes)
            #the below giving dictionary full of blanks
            uasOd[z] = (flightTimes, flightTracks,flightAltitudes)
            #uasNumberedDictionary[z + str(heatIterator)] = 
            self.uasOd = uasOd
            
            

            
    def refreshUasDisplay(self):
        
        
        lp = 0
        lines = []
        colorRange = 255/len(self.uasFlights)

        for i in self.uasFlights:
            if lp == 0:
                colorIn = self.RGBtoHex([255,0,0])
            else:
                colorIn = self.RGBtoHex([ 255 , 0, int(255-(colorRange * lp)) ])

            
            line = Polyline(
            locations = 
                [[
                    self.uasOd[i][1][lp]
                ,]],
                color = colorIn,
                fill_color = colorIn,
                fill = False,
                opacity = .7)

            line.popup=HTML(i)
            lines.append(line)


            lp+=1;
            
        self.uasLines = lines
        if self.uasDisplay == None:
            self.uasDisplay = LayerGroup(layers=(lines)) 
        else:
            self.uasDisplay.layers = self.uasLines
             
    def refreshUasHeatTree(self, heatDistanceDelimiter):
        
        
        self.heatDistanceDelimiter = heatDistanceDelimiter
        
        if self.heatDisplay == None:
            circles = []

            iterator = 0
            uasTree =  qt.Index(bbox= (-90, -180, 90, 180))
            for uasFlight in self.uasFlights:
                uasTreeCoordIndex = 0
                for coordinate in self.uasOd[uasFlight][1][iterator]:

                    latitudeFeetConversion = 6076 * 60
                    
                    xmin = ((coordinate[0] * 6076)*60 - self.heatDistanceDelimiter) / latitudeFeetConversion
                    xmax = ((coordinate[0] * 6076)*60 + self.heatDistanceDelimiter) / latitudeFeetConversion
                    ymin = ((coordinate[1] * 6076)*60 - (self.heatDistanceDelimiter * math.cos(math.radians(xmin)))) / latitudeFeetConversion
                    ymax = ((coordinate[1] * 6076)*60 + (self.heatDistanceDelimiter * math.cos(math.radians(xmax)))) / latitudeFeetConversion
                    self.uasHeat[uasFlight + str(uasTreeCoordIndex)] = (xmin, ymin, xmax, ymax)

                    uasTree.insert(uasFlight + str(uasTreeCoordIndex), self.uasHeat[uasFlight + str(uasTreeCoordIndex)])
                    uasTreeCoordIndex+=1

                    heatCircle = Circle()
                    heatCircle.location = coordinate
                    heatCircle.radius = int(self.heatDistanceDelimiter / 3)
                    heatCircle.popup = HTML(uasFlight + '\n waypt#' +str(uasTreeCoordIndex))
                    heatCircle.color = 'darkred'
                    heatCircle.opacity = .3
                    heatCircle.weight = 1
                    heatCircle.fill_color='darkred'
                    circles.append(heatCircle)

                iterator+=1

            self.heatCircles = circles
            self.uasTree = uasTree
            self.heatDisplay = LayerGroup(layers=(circles)) 
        else:
            iterator = 0
            uasTree =  qt.Index(bbox= (-90, -180, 90, 180))
            for uasFlight in self.uasFlights:
                uasTreeCoordIndex = 0
                for coordinate in self.uasOd[uasFlight][1][iterator]:

                    latitudeFeetConversion = 6076 * 60
                    
                    xmin = ((coordinate[0] * 6076)*60 - self.heatDistanceDelimiter) / latitudeFeetConversion
                    xmax = ((coordinate[0] * 6076)*60 + self.heatDistanceDelimiter) / latitudeFeetConversion
                    ymin = ((coordinate[1] * 6076)*60 - (self.heatDistanceDelimiter * math.cos(math.radians(xmin)))) / latitudeFeetConversion
                    ymax = ((coordinate[1] * 6076)*60 + (self.heatDistanceDelimiter * math.cos(math.radians(xmax)))) / latitudeFeetConversion
                    self.uasHeat[uasFlight + str(uasTreeCoordIndex)] = (xmin, ymin, xmax, ymax)

                    uasTree.insert(uasFlight + str(uasTreeCoordIndex), self.uasHeat[uasFlight + str(uasTreeCoordIndex)])
                    uasTreeCoordIndex+=1

                iterator+=1
            self.uasTree = uasTree  
            for heatCircle in self.heatCircles:
                heatCircle.radius = int(self.heatDistanceDelimiter / 3.281)
            self.heatDisplay.layers = self.heatCircles
        ;
        
        
    def refreshAdsbTree(self):
        

        adsbTree = qt.Index(bbox= (-90, -180, 90, 180))

        for flight in self.adsbFlights:
            flightTreeIndex = 0
            for coordinates in self.adsbFlightsDictionary[flight + 'track']:
                adsbTree.insert(flight + ',' + str(flightTreeIndex), (coordinates[0], coordinates[1], coordinates[0], coordinates[1]))
                flightTreeIndex+=1
        self.adsbTree = adsbTree
        
    def burnEncounterTrees(self):
        
        encounterResults = []
        uasEncounters = []

        for uasHeat in self.uasHeat:

            result = self.adsbTree.intersect(self.uasHeat[uasHeat])
            if len(result) > 0:

                encounterResults.append(result)
                uasEncounters.append([result,uasHeat[:-1], uasHeat[-1]])
            
        self.encounterResults = encounterResults
        self.uasEncounters = uasEncounters
        
        encounterFlights = []
        uasEncounterFlights = []
        
        for uasEncounterList in self.encounterResults:
            for encounter in uasEncounterList:
                encounter = encounter.split(',')
                if encounter[0] in encounterFlights:
                    ;
                else:
                    encounterFlights.append(encounter[0])

        self.encounterFlights = encounterFlights
        ;

    def refreshEncounterDisplay(self):
        oldColors = []
        colorShuffle = 0
        for name in self.adsbFlights:

            if self.adsbLines[self.adsbFlights.index(name)].color == 'gold':
                self.adsbLines[self.adsbFlights.index(name)].color = self.oldColors[colorShuffle][0]
                colorShuffle +=1

            if name in self.encounterFlights:
                oldColors.append((self.adsbLines[self.adsbFlights.index(name)].color, self.adsbFlights.index(name)))
                self.adsbLines[self.adsbFlights.index(name)].color = 'gold'
            else:
                ;
        self.oldColors = oldColors
        
        
    def altitudeCompare(self, altitudeDelimiter):
        
        self.altitudeDelimiter = altitudeDelimiter
        
        newEncounters = []
        
        for i in list(self.uasEncounters[:]):
            altLow = min(self.uasFlights[i[1][:14]][2]) * 3.28084 - self.altitudeDelimiter/2
            altHigh = max(self.uasFlights[i[1][:14]][2]) * 3.28084 + self.altitudeDelimiter/2
            
            for z in i[:][0]:
                if (self.adsbFlightsDictionary[z.split(',')[0]+'altitude']
                [int(z.split(',')[1])]
                [1]) >altLow and (self.adsbFlightsDictionary[z.split(',')[0]+'altitude']
                [int(z.split(',')[1])]
                [1]) < altHigh:
                    ;
                    #print(self.adsbFlightsDictionary[z.split(',')[0]+'altitude']
                          #[int(z.split(',')[1])])
                    #print('pass')

                    #print(mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'altitude']
        #[int(z.split(',')[1])]
        #[1])
                    #print(altLow)
                    #print(altHigh)
                else:
                    #print(self.adsbFlightsDictionary[z.split(',')[0]+'altitude']
                    #      [int(z.split(',')[1])])
                    self.uasEncounters.i[0].remove(z)
                    #print('removed', z , "from" , i[1][:14] + i[2])
                    #print(' ')
                    #i[0].remove(z)
                    
#         for uasEncounterList in self.encounterResults:
#             for encounter in uasEncounterList:
#                 encounter = encounter.split(',')
#                 if encounter[0] in encounterFlights:
#                     ;
#                 else:
#                     encounterFlights.append(encounter[0])

#         self.encounterFlights = encounterFlights
            
        

        
        
        
    def timeCompare(self, timeDelimiter):
        
        self.timeDelimiter = timeDelimiter
        
        
        

    def rebuildDf(self):
        
        
        self.heatDisplay.clear_layers()
        self.heatDisplay = None        
        self.uasHeat = {}
        self.adsbTree = {}
        self.encounterList = None
        self.buildAdsbFlights()
        self.refreshAdsbFlights()
        self.refreshAdsbDisplay()
        self.refreshUasFlights()
        self.refreshUasDisplay()
        self.refreshAdsbTree()
        self.refreshUasHeatTree(0)
        self.burnEncounterTrees()
        self.refreshEncounterDisplay()
        
        if self.mapp == None:
            mapp.add_layer(self.heatDisplay)
        else:
            self.mapp.add_layer(self.heatDisplay)
            

        

In [959]:
class mappConstructor():
    def __init__(self):
        
        self.mdf = None
        self.airport = 'DAB'
        self.mapp = None
        self.zoom = 10
        self.dataType = ['ADSB', 'AERO', 'TFMS']
        self.dataDate = ['2019', '08', '24']
        self.altitudeLow = 0
        self.altitudeHigh = 4000
        self.time = [' 00:00:00', ' 23:59:59']
        self.encounterDisplay = None
        self.adsbSelector = None
        self.aeroSelector = None
        self.encounterList = None
        self.altitudeDisplay = None
        
    def buildDataFrame(self):
            
        
        mdf = mappDataFrame()
        mdf.buildDataFrames(self.dataType, self.dataDate, self.airport)
        mdf.refreshParameters(self.altitudeLow ,self.altitudeHigh, self.time)
        mdf.buildAdsbFlights()
        mdf.refreshAdsbFlights()
        mdf.refreshAdsbDisplay()
        mdf.refreshUasFlights()
        mdf.refreshUasDisplay()
        mdf.refreshAdsbTree()
        mdf.refreshUasHeatTree(0)
        mdf.burnEncounterTrees()
        mdf.refreshEncounterDisplay()
        
        self.mdf = mdf
        
    def construct(self):
        
        
        
        mapp = Map(
    layers=(basemap_to_tiles(basemaps.Stamen.Toner), ),
        zoom = self.zoom,
        )
        
        mapp.add_layer(self.mdf.adsbDisplay)
        mapp.add_layer(self.mdf.uasDisplay)
        mapp.add_layer(self.mdf.heatDisplay)
        mapp.add_control(FullScreenControl()) 
        
        if self.airport == 'DAB':
            mapp.center = [29.1832, -81.0532]
            
        elif self.airport == 'ABQ':
            mapp.center = [35.0433, -106.6129]
            
        self.mapp = mapp
        self.addWidgets()
        self.mdf.mapp = self.mapp
        
    def display(self):
        
        return self.mapp
    
    def rebuildMdf(self, altitudeLow, altitudeHigh, timeLow, timeHigh):
    
        self.altitudeLow = altitudeLow
        self.altitudeHigh = altitudeHigh
        self.time = [timeLow, timeHigh]
        
        
        
        self.mdf.refreshParameters(self.altitudeLow, self.altitudeHigh, self.time)
        self.mdf.rebuildDf()
    
    def refreshEncounterDistance(self, heatDistanceDelimiter):
    
        self.mdf.refreshUasHeatTree(heatDistanceDelimiter)
        self.mdf.burnEncounterTrees()
        self.mdf.refreshEncounterDisplay()

        
    def refreshEncounters(self, altitudeDelimiter):
        
        self.mdf.compareAltitude(altitudeDelimiter)
        
        
        
    def addWidgets(self):
        
        encounterDisplay = widgets.BoundedIntText(
            value = 0,
            min = 0,
            max = 1000,
            step = 20,
            description = 'encounterDistance',
            disabled = False,
            continuous_update=True
        )

        encounterOutput = widgets.Output()

        def encounterChange(change):
            with encounterOutput:
                self.refreshEncounterDistance(encounterDisplay.value)
                self.encounterList.options = self.mdf.encounterFlights
                print(encounterDisplay.value)

        
        encounterDisplay.observe(encounterChange, names='value')
        self.encounterDisplay = encounterDisplay
        
        adsbSelector = widgets.SelectMultiple(
            options = self.mdf.adsbFlights,
            value = self.mdf.adsbFlights[0:2],
            description = 'adsbFlights',
            continuous_update = True)

        adsbFlightOutput = widgets.Output()

        def adsbChange(change):
            with adsbFlightOutput:
                self.mdf.adsbDisplay.clear_layers()
                for x in adsbSelector.options:
                    if ( str(x) in adsbSelector.value ):
                        self.mdf.adsbDisplay.add_layer(
                            self.mdf.adsbLines[self.mdf.adsbFlights.index(x)])     
                    else:
                        ;
        adsbSelector.observe(adsbChange, names = 'value')
        self.adsbSelector = adsbSelector

 
        
        aeroSelector = widgets.SelectMultiple(
        options = list(self.mdf.uasFlights.keys()),
        value = list(self.mdf.uasFlights.keys())[0:2],
        description = 'djiaeroscope',
        continuous_update = True)

        aeroFlightOutput = widgets.Output()
        
        def aeroChange(change):
            with aeroFlightOutput:
                self.mdf.uasDisplay.clear_layers()
                for x in aeroSelector.options:
                    if ( str(x) in aeroSelector.value ):
                        self.mdf.uasDisplay.add_layer(
                            self.mdf.uasLines[list(self.mdf.uasFlights.keys()).index(x)])     
                    else:
                        ;
                
        
        aeroSelector.observe(aeroChange, names='value')
        self.aeroSelector = aeroSelector
        
               
        encounterList = widgets.Dropdown(options = self.mdf.encounterFlights, 
                                         value=None, 
                                         continuous_update=True,
                                         description = 'encounterList')
        
        self.encounterList = encounterList
        
        altitudeDisplay = widgets.BoundedIntText(
            value = 0,
            min = 0,
            max = 1000,
            step = 20,
            description = 'altitudeDistance',
            disabled = False,
            continuous_update=True
        )
        
        altitudeOutput = widgets.Output()
        
        def altitudeChange(change):
            
            with altitudeOutput:
                
                self.refreshEncounters(altitudeDisplay.value)
                #print(altitudeDisplay.value)
                
        altitudeDisplay.observe(altitudeChange, names = 'value')
        self.altitudeDisplay = altitudeDisplay
        
        
#         encounterListOutput = widgets.Output()
        
#         def encounterListChange(change):
#             with encounterListOutput:
#                 self.mdf.uasDisplay.clear_layers()
#                 for x in aeroSelector.options:
#                     if ( str(x) in aeroSelector.value ):
#                         self.mdf.uasDisplay.add_layer(
#                             self.mdf.uasLines[list(self.mdf.uasFlights.keys()).index(x)])     
#                     else:
#                         ;
                
        
        
        accordion = widgets.Accordion(children=[adsbSelector,
                                                encounterDisplay,
                                                aeroSelector,
                                                altitudeDisplay])
        accordion.set_title(0, 'adsb')
        accordion.set_title(1, 'encounter')
        accordion.set_title(2, 'aeroscope')
        accordion.set_title(3, 'altitude')
    
        
        widgetControl = WidgetControl(widget = accordion, position = 'bottomright')
        widgetControlLeft = WidgetControl(widget = encounterList, position = 'bottomleft')
        self.mapp.add_control(widgetControl)
        self.mapp.add_control(widgetControlLeft)
    

In [960]:
mapp = mappConstructor()
mapp.buildDataFrame()

In [961]:
mapp.construct()

In [962]:
mapp.display()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [971]:
mapp.mdf.uasEncounters

# for i in list(self.uasEncounters[:]):
#     altLow = min(self.uasFlights[i[1][:14]][2]) * 3.28084 - self.altitudeDelimiter/2
#     altHigh = max(self.uasFlights[i[1][:14]][2]) * 3.28084 + self.altitudeDelimiter/2

#     for z in i[:][0]:
#         if (self.adsbFlightsDictionary[z.split(',')[0]+'altitude']
#         [int(z.split(',')[1])]
#         [1]) >altLow and (self.adsbFlightsDictionary[z.split(',')[0]+'altitude']
#         [int(z.split(',')[1])]
#         [1]) < altHigh:
#             ;
#             #print(self.adsbFlightsDictionary[z.split(',')[0]+'altitude']
#                   #[int(z.split(',')[1])])
#             #print('pass')

#             #print(mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'altitude']
# #[int(z.split(',')[1])]
# #[1])
#             #print(altLow)
#             #print(altHigh)
#         else:
#             #print(self.adsbFlightsDictionary[z.split(',')[0]+'altitude']
#             #      [int(z.split(',')[1])])
#             self.uasEncounters.i[0].remove(z)

[[['N1230G,446', 'N1230G,445'], '163DFAS00180PB', '0'],
 [['N50317,426', 'N50317,427'], '163DFAS00180PB', '1'],
 [['N431RP,292', 'N431RP,293'], '163DFAS00180PB', '3'],
 [['N431RP,292', 'N431RP,293'], '163DFAS00180PB', '4'],
 [['N536ND,440', 'N431RP,289', 'N431RP,290'], '163DFAS00180PB', '8'],
 [['N536ND,442', 'N20571,11', 'N536ND,443', 'N20571,12'],
  '163DFAS00180PB',
  '9'],
 [['N20571,15', 'N536ND,446', 'N536ND,444', 'N536ND,445', 'N20571,14'],
  '163DFAS00180PB1',
  '0'],
 [['N50317,425', 'N50317,426', 'N50317,427'], '163DFAS00180PB1', '1'],
 [['N8480J,281', 'N8480J,282'], '08RDE3V001017Q', '3'],
 [['N8480J,281', 'N8480J,282', 'N8480J,283'], '08RDE3V001017Q', '4'],
 [['N8480J,281', 'N8480J,282', 'N8480J,283'], '08RDE3V001017Q', '5'],
 [['N8480J,281', 'N8480J,282', 'N8480J,283'], '08RDE3V001017Q', '6'],
 [['N8480J,281', 'N8480J,282', 'N8480J,283'], '08RDE3V001017Q', '7'],
 [['N8480J,282', 'N8480J,283'], '08RDE3V001017Q', '8'],
 [['N8480J,282', 'N8480J,283'], '08RDE3V001017Q', '9']]

In [970]:
iterator = 0
uasTree =  qt.Index(bbox= (-90, -180, 90, 180))
for uasFlight in mapp.mdf.uasFlights:
    uasTreeCoordIndex = 0
    for coordinate in mapp.mdf.uasOd[uasFlight][2][iterator]:

        ;
        #print(coordinate * 3)
        #print(uasFlight)
    for coordinate in mapp.mdf.uasOd[uasFlight][0][iterator]:
             
            
        print(coordinate)
        
        #datetime_object = datetime.strptime(coordinate, '%Y-%m-%d %H:%M:%S')
        #print( datetime_object + timedelta(hours=6))
        
        #print(uasFlight)
    for coordinate in mapp.mdf.uasOd[uasFlight][1][iterator]:
        
        ;
        #print(coordinate)
        #print(uasFlight)
    iterator+=1

2019-08-24 22:46:56
2019-08-24 22:46:52
2019-08-24 22:46:46
2019-08-24 22:46:40
2019-08-24 22:46:38
2019-08-24 22:46:38
2019-08-24 22:46:36
2019-08-24 22:46:32
2019-08-24 22:46:32
2019-08-24 22:46:30
2019-08-24 22:46:26
2019-08-24 22:46:26
2019-08-24 22:46:24
2019-08-24 22:46:24
2019-08-24 22:46:24
2019-08-24 22:46:22
2019-08-24 22:46:22
2019-08-24 22:46:20
2019-08-24 22:46:20
2019-08-24 22:46:20
2019-08-24 22:46:16
2019-08-24 22:46:14
2019-08-24 22:46:12
2019-08-24 22:46:08
2019-08-24 22:46:06
2019-08-24 22:46:04
2019-08-24 22:46:04
2019-08-24 22:46:00
2019-08-24 22:45:58
2019-08-24 22:45:56
2019-08-24 22:45:54
2019-08-24 22:45:48
2019-08-24 22:45:46
2019-08-24 22:45:42
2019-08-24 22:45:40
2019-08-24 22:45:36
2019-08-24 22:45:34
2019-08-24 22:45:32
2019-08-24 22:45:28
2019-08-24 22:45:26
2019-08-24 22:45:20
2019-08-24 22:45:14
2019-08-24 22:45:12
2019-08-24 22:45:08
2019-08-24 22:45:06
2019-08-24 22:44:54
2019-08-24 22:44:52
2019-08-24 22:44:36
2019-08-24 22:43:13
2019-08-24 22:43:11


In [ ]:
mapp.rebuildMdf(0, 2000, ' 06:00:00', ' 24:00:00')

In [462]:
#mapp.mdf.uasEncounters
#mapp.refreshEncounterDistance(0)

mapp.mdf.uasFlights['163DF84001KSD8'][0][0]

#['08RDE620010192']

# '08RDE6Q00102N9'
# '0K1CF5S3AE1EMT'
# '07JDEB600201L6'
# '163DF810013D1X'
# '163DF84001KSD8'



#popo = list(mapp.mdf.uasFlights.keys())

24    29.251232
23    29.251232
22    29.251232
21    29.251232
20    29.251232
19    29.251232
18    29.251232
17    29.251581
16    29.253523
15    29.253523
14    29.253518
13    29.253518
12    29.253472
11    29.252819
10    29.252286
9     29.251942
8     29.251186
7     29.251186
6     29.251180
5     29.250109
4     29.250034
3     29.249358
2     29.249335
1     29.249324
0     29.249324
Name: Latitude, dtype: float64

In [65]:
list(mapp.mdf.uasOd['163DFAS00180PB'])

[[[],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  ['2019-08-24 20:02:16',
   '2019-08-24 20:01:24',
   '2019-08-24 20:01:24',
   '2019-08-24 20:01:16',
   '2019-08-24 20:01:10',
   '2019-08-24 20:01:08',
   '2019-08-24 20:00:52',
   '2019-08-24 20:00:36',
   '2019-08-24 20:00:28',
   '2019-08-24 20:00:10',
   '2019-08-24 20:00:02',
   '2019-08-24 19:59:58',
   '2019-08-24 19:58:56'],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  ['2019-08-24 21:48:24', '2019-08-24 21:45:33', '2019-08-24 21:44:51'],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  ['2019-08-24 16:46:56',
   '2019-08-24 16:46:52',
   '2019-08-24

In [12]:
mapp.mdf.encounterFlights

['N62568',
 'N5144H',
 'N882TC',
 'N524ND',
 'N50317',
 'N565AD',
 'RDDL188',
 'N8480J',
 'RDDL187',
 'N714HW',
 'N183BL',
 'N20571',
 'N536ND',
 'N5280R',
 'RDDL189',
 'N431RP',
 'N13BF',
 'N143CC',
 'N2504M',
 'N35HW',
 'RDDL483',
 'N559ND',
 'RDDL490',
 'N1448Z',
 'DAL771',
 'N52246',
 'RDDL942',
 'N1159K',
 'RDDL491',
 'RDDL414',
 'RDDL475',
 'N1230G',
 'N5526J',
 'N2459M',
 'CONN463']

In [11]:
out = mapp.mdf.encounterResults[5][0].split(',')[1]

IndexError: list index out of range

In [274]:
for x in mapp.encounterList.options:
    
    for y in mapp.mdf.encounterResults:
        for i in y[:]:
            if i.split(',')[0] == x:
                
                print(x)
                print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'altitude'][int(i.split(',')[1])])
                print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])])
                print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'track'][int(i.split(',')[1])])
                
                



N8480J
['AirBorn', 725]
2019-08-24 16:42:08
[29.158916473388672, -80.97198486328125]
N8480J
['AirBorn', 725]
2019-08-24 16:42:11
[29.157886505126953, -80.9717059135437]
N8480J
['AirBorn', 725]
2019-08-24 16:42:10
[29.158272743225094, -80.97177028656006]
N8480J
['AirBorn', 725]
2019-08-24 16:42:10
[29.158079624176025, -80.97177028656006]
N8480J
['AirBorn', 725]
2019-08-24 16:42:09
[29.15868043899536, -80.97187757492065]
N8480J
['AirBorn', 725]
2019-08-24 16:42:09
[29.15848731994629, -80.9718132019043]
N8480J
['AirBorn', 800]
2019-08-24 16:40:26
[29.19835567474365, -80.99239110946654]
N8480J
['AirBorn', 800]
2019-08-24 16:40:26
[29.19835567474365, -80.99239110946654]
N8480J
['AirBorn', 800]
2019-08-24 16:40:26
[29.19835567474365, -80.99239110946654]
N8480J
['AirBorn', 800]
2019-08-24 16:40:26
[29.19816255569458, -80.99234819412231]
N8480J
['AirBorn', 800]
2019-08-24 16:40:25
[29.19891357421875, -80.99271297454834]
N8480J
['AirBorn', 800]
2019-08-24 16:40:25
[29.19867753982544, -80.992584

IndexError: list index out of range

In [57]:
len(mapp.mdf.encounterResults)

20

In [58]:
mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'altitude'][292]

['AirBorn', 975]

In [60]:
x = mapp.encounterList.value
for y in mapp.mdf.encounterResults:
    for i in y[:]:
        if i.split(',')[0] == x:
            print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'altitude'][int(i.split(',')[1])])
            print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])])
            #timerr = mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])]
            #print(timerr)
            #datetime_object = datetime.strptime(timerr, '%Y-%m-%d %H:%M:%S')
            print( datetime_object + timedelta(hours=6))
            
            print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'track'][int(i.split(',')[1])])

['AirBorn', 1850]
2019-08-24 17:47:18
2019-08-24 23:47:21
[29.1806960105896, -80.9837007522583]
['AirBorn', 1850]
2019-08-24 17:47:19
2019-08-24 23:47:21
[29.180567264556885, -80.98344326019287]
['AirBorn', 1875]
2019-08-24 17:47:20
2019-08-24 23:47:21
[29.18033123016357, -80.98290681838989]
['AirBorn', 1875]
2019-08-24 17:47:21
2019-08-24 23:47:21
[29.18018102645874, -80.98264932632445]


In [ ]:
mapp.mdf.uasHeat

In [69]:
for uasHeat in mapp.mdf.uasHeat:
    print(uasHeat[-1:])

0
1
2
3
4
5
6
7
8
9
0
1
2
0
1
2
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
0
0
1
2
3
4
5
6
7
8
9
0
1
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
0
1
0
1
2
3
4
5
6
7
8
9
0
1
0
1
0


In [88]:
p = 0
for x in mapp.mdf.uasOd['08RDE8F001021K']:
    if len(x)>1:
        print(x)

#mapp.mdf.uasOd['08RDE8F001021K'][2]

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['2019-08-24 20:02:16', '2019-08-24 20:01:24', '2019-08-24 20:01:24', '2019-08-24 20:01:16', '2019-08-24 20:01:10', '2019-08-24 20:01:08', '2019-08-24 20:00:52', '2019-08-24 20:00:36', '2019-08-24 20:00:28', '2019-08-24 20:00:10', '2019-08-24 20:00:02', '2019-08-24 19:59:58', '2019-08-24 19:58:56'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['2019-08-24 21:48:24', '2019-08-24 21:45:33', '2019-08-24 21:44:51'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['2019-08-24 16:46:56', '2019-08-24 16:46:52', '2019-08-24 16:46:46', '2019-08-24 16:46:40', '2019-08-24 16:46:38', '2019-08-24 16:46:38', '2019-08-24 16:46:36', '2019-08-24 16:46:32', '2019-08-24 16:46:32', '2019-08-24 16:46:30', '2019-08-24 16:46:26', '2019-08-24 16:46:26', '2

In [89]:
mapp.mdf.uasEncounters[0][1]

'163DFAS00180PB'

In [91]:
#mapp.mdf.uasOd['163DFAS00180PB']

In [44]:
#mapp.mdf.uasOd['07JDE74002005F']

In [45]:
#{k: v for k, v in mapp.mdf.uasOd.items() if v}

In [275]:
#len(mapp.mdf.uasOd['0K1UF6600F0070'][0][:])
mapp.mdf.uasOd['0K1UF6600F0070'][0][:]

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['2019-08-24 22:46:56',
  '2019-08-24 22:46:52',
  '2019-08-24 22:46:46',
  '2019-08-24 22:46:40',
  '2019-08-24 22:46:38',
  '2019-08-24 22:46:38',
  '2019-08-24 22:46:36',
  '2019-08-24 22:46:32',
  '2019-08-24 22:46:32',
  '2019-08-24 22:46:30',
  '2019-08-24 22:46:26',
  '2019-08-24 22:46:26',
  '2019-08-24 22:46:24',
  '2019-08-24 22:46:24',
  '2019-08-24 22:46:24',
  '2019-08-24 22:46:22',
  '2019-08-24 22:46:22',
  '2019-08-24 22:46:20',
  '2019-08-24 22:46:20',
  '2019-08-24 22:46:20',
  '2019-08-24 22:46:16',
  '2019-08-24 22:46:14',
  '20

In [43]:
uasFlight = '0K1UF6600F0070'
mapp.mdf.uasOd[uasFlight][1][15]

[]

In [276]:
iterator = 0
oldlist = []
uasTree =  qt.Index(bbox= (-90, -180, 90, 180))
for uasFlight in mapp.mdf.uasFlights:
    uasTreeCoordIndex = 0
    for coordinate in mapp.mdf.uasOd[uasFlight][0][iterator]:
        
        oldlist.append(coordinate)
        print(coordinate)
        print(uasFlight)
        
        
    
    iterator+=1

2019-08-24 22:46:56
07JDE4R00A01G6
2019-08-24 22:46:52
07JDE4R00A01G6
2019-08-24 22:46:46
07JDE4R00A01G6
2019-08-24 22:46:40
07JDE4R00A01G6
2019-08-24 22:46:38
07JDE4R00A01G6
2019-08-24 22:46:38
07JDE4R00A01G6
2019-08-24 22:46:36
07JDE4R00A01G6
2019-08-24 22:46:32
07JDE4R00A01G6
2019-08-24 22:46:32
07JDE4R00A01G6
2019-08-24 22:46:30
07JDE4R00A01G6
2019-08-24 22:46:26
07JDE4R00A01G6
2019-08-24 22:46:26
07JDE4R00A01G6
2019-08-24 22:46:24
07JDE4R00A01G6
2019-08-24 22:46:24
07JDE4R00A01G6
2019-08-24 22:46:24
07JDE4R00A01G6
2019-08-24 22:46:22
07JDE4R00A01G6
2019-08-24 22:46:22
07JDE4R00A01G6
2019-08-24 22:46:20
07JDE4R00A01G6
2019-08-24 22:46:20
07JDE4R00A01G6
2019-08-24 22:46:20
07JDE4R00A01G6
2019-08-24 22:46:16
07JDE4R00A01G6
2019-08-24 22:46:14
07JDE4R00A01G6
2019-08-24 22:46:12
07JDE4R00A01G6
2019-08-24 22:46:08
07JDE4R00A01G6
2019-08-24 22:46:06
07JDE4R00A01G6
2019-08-24 22:46:04
07JDE4R00A01G6
2019-08-24 22:46:04
07JDE4R00A01G6
2019-08-24 22:46:00
07JDE4R00A01G6
2019-08-24 22:45:58


In [575]:
iterator = 0
oldlist = []
oldlist2 = []
oldlist3 = []
uasTree =  qt.Index(bbox= (-90, -180, 90, 180))
for uasFlight in mapp.mdf.uasFlights:
    uasTreeCoordIndex = 0
    for coordinate in mapp.mdf.uasOd[uasFlight][0][iterator]:
        
        oldlist.append(coordinate)
        #print(coordinate)
        #print(uasFlight)
        #print(iterator)
        #iterator=+1
        
    for coordinate in mapp.mdf.uasOd[uasFlight][1][iterator]:
        
        oldlist2.append(coordinate)
        #print(coordinate)
        #print(uasFlight)
        
        
    for coordinate in mapp.mdf.uasOd[uasFlight][2][iterator]:
        
        oldlist3.append(coordinate*3)
        #print(coordinate)
        print(uasFlight)
        
        
    
    iterator+=1
    
    
print(oldlist2)

07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A01G6
07JDE4R00A

In [277]:

uasFlight = '0K1UF6600F0070'
iterator= 0

for coordinate in mapp.mdf.uasOd[uasFlight][0][iterator]:

    oldlist.append(coordinate)
    print(coordinate)
    print(uasFlight)
    #print(iterator)
    #iterator=+1

for coordinate in mapp.mdf.uasOd[uasFlight][1][iterator]:

    oldlist2.append(coordinate)
    print(coordinate)
    print(uasFlight)


for coordinate in mapp.mdf.uasOd[uasFlight][2][iterator]:

    oldlist3.append(coordinate*3)
    print(coordinate)
    print(uasFlight)

    #iterator+=1
    

In [278]:
oldlist3
#mapp.mdf.uasEncounters[0][-2:-1]

[201.60000000000002,
 295.79999999999995,
 355.5,
 378.29999999999995,
 378.6,
 378.29999999999995,
 377.4,
 413.09999999999997,
 413.09999999999997,
 413.09999999999997,
 398.40000000000003,
 406.79999999999995,
 398.40000000000003,
 398.40000000000003,
 398.40000000000003,
 398.40000000000003,
 398.40000000000003,
 374.1,
 388.5,
 355.79999999999995,
 356.1,
 356.1,
 355.5,
 330.9,
 330.9,
 330.9,
 330.9,
 304.5,
 291.0,
 291.0,
 277.20000000000005,
 249.89999999999998,
 250.20000000000002,
 222.60000000000002,
 222.60000000000002,
 185.39999999999998,
 185.39999999999998,
 171.89999999999998,
 162.60000000000002,
 162.60000000000002,
 161.39999999999998,
 161.10000000000002,
 161.10000000000002,
 160.8,
 160.8,
 160.8,
 161.10000000000002,
 161.10000000000002,
 162.60000000000002,
 162.60000000000002,
 162.0,
 161.7,
 161.7,
 162.0,
 161.7,
 161.7,
 161.39999999999998,
 163.8,
 163.8,
 163.2,
 162.89999999999998,
 163.2,
 163.2,
 163.5,
 142.2,
 166.5,
 166.5,
 176.10000000000002,
 

In [307]:
o=0
for i in mapp.mdf.uasEncounters:
    
    
    x = i[-2:][:14]
    #y = i[14:]
    #x = i[-2:][0][:14].split(',')
    #print(i[-2:][0][:14])
    #print(i[-2:][1])
    o+=1
    print(x)
    
    
print(o)

print(len('08RDEA200101WW'))

['163DFAS00180PB', '0']
['163DFAS00180PB', '1']
['163DFAS00180PB', '2']
['163DFAS00180PB', '3']
['163DFAS00180PB', '4']
['163DFAS00180PB', '5']
['163DFAS00180PB', '6']
['163DFAS00180PB', '7']
['163DFAS00180PB', '8']
['163DFAS00180PB', '9']
['163DFAS00180PB1', '0']
['163DFAS00180PB1', '1']
['08RDE3V001017Q', '0']
['08RDE3V001017Q', '1']
['08RDE3V001017Q', '2']
['08RDE3V001017Q', '3']
['08RDE3V001017Q', '4']
['08RDE3V001017Q', '5']
['08RDE3V001017Q', '6']
['08RDE3V001017Q', '7']
['08RDE3V001017Q', '8']
['08RDE3V001017Q', '9']
22
14


In [62]:
o = 0
for i in mapp.mdf.encounterFlights:
    print(i)
    o+=1
print(o)

N62568
N5144H
N882TC
N7YJ
N524ND
N50317
N565AD
RDDL362
N8480J
RDDL188
N5280R
N183BL
RDDL189
RDDL187
N714HW
N5526J
N536ND
N20571
N1230G
N431RP
N13BF
N143CC
N2504M
RDDL483
N35HW
N10624
N818EP
N550ND
N559ND
RDDL490
N1448Z
DAL771
ENY9800
JIA5545
JIA5601
JIA5315
N975DM
RPA4620
JIA5175
N52246
RDDL942
N1159K
RDDL491
RDDL414
RDDL475
N487DA
N2459M
CONN463
48


In [79]:
mapp.mdf.uasEncounters[0]

[['N20571,38',
  'N20571,39',
  'N1230G,446',
  'N1230G,447',
  'N1230G,444',
  'N1230G,445',
  'N20571,40',
  'N20571,41',
  'N1230G,443',
  'N20571,42'],
 '163DFAS00180PB',
 '0']

In [61]:
mapp.mdf.encounterFlights[0]

'N20571'

In [60]:
mapp.mdf.encounterResults[0]

['N20571,38',
 'N20571,39',
 'N1230G,446',
 'N1230G,447',
 'N1230G,444',
 'N1230G,445',
 'N20571,40',
 'N20571,41',
 'N1230G,443',
 'N20571,42']

In [75]:
mapp.mdf.uasOd['163DFAS00180PB'][1][1]

[]

In [165]:
iterator = 0
uasTree =  qt.Index(bbox= (-90, -180, 90, 180))
for uasFlight in mapp.mdf.uasFlights:
    uasTreeCoordIndex = 0
    for coordinate in mapp.mdf.uasOd[uasFlight][2][iterator]:

        ;
        #print(coordinate * 3)
        #print(uasFlight)
    for coordinate in mapp.mdf.uasOd[uasFlight][0][iterator]:
        
        print(coordinate)
        
#         datetime_object = datetime.strptime(coordinate, '%Y-%m-%d %H:%M:%S')
#         print( datetime_object + timedelta(hours=6))
        
#         print(uasFlight)
    for coordinate in mapp.mdf.uasOd[uasFlight][1][iterator]:
        
        ;
        #print(coordinate)
        #print(uasFlight)
    iterator+=1

2019-08-24 02:47:56
2019-08-24 08:43:42
2019-08-24 08:43:32
2019-08-24 08:42:46
2019-08-24 22:02:18
2019-08-24 22:26:35
2019-08-24 22:26:05
2019-08-24 22:25:49
2019-08-24 22:25:45
2019-08-24 22:25:43
2019-08-24 22:25:39
2019-08-24 22:25:33
2019-08-24 22:25:33
2019-08-24 22:25:27
2019-08-24 22:25:25
2019-08-24 22:25:19
2019-08-24 22:24:57


In [61]:
newlist = list(mapp.mdf.uasEncounters[0])

In [62]:
newlist[0][12].split(',')[0]

'N1230G'

In [898]:
#newlist = list(mapp.mdf.uasEncounters[8])
#newlist

print(mapp.mdf.uasEncounters[20])

[['N559ND,709', 'N559ND,713', 'N559ND,699', 'N559ND,700', 'N559ND,704'], '07JDE3300201B0', '7']


In [109]:
#mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'altitude'][292]
mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][292]
#mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'track'][292]

# C = [1,2,34]
# D = True
# di = [B*B for B in C if D]

'2019-08-24 17:44:34'

In [47]:
di

[1, 4, 1156]

In [190]:
mapp.mdf.uasFlights

{'08RDE620010192': [[5    29.194188
   4    29.194188
   3    29.194371
   2    29.194446
   1    29.194463
   0    29.194463
   Name: Latitude, dtype: float64, 5   -81.047968
   4   -81.047963
   3   -81.047516
   2   -81.047384
   1   -81.047332
   0   -81.047332
   Name: Longitude, dtype: float64], 5    2019-08-14 15:21:14
  4    2019-08-14 15:21:40
  3    2019-08-14 15:23:12
  2    2019-08-14 15:23:26
  1    2019-08-14 15:25:34
  0    2019-08-14 15:25:36
  Name: TimeWhen, dtype: object, 5    0.0
  4    0.0
  3    0.0
  2    0.0
  1    0.0
  0    0.0
  Name: IconAltitude, dtype: float64], '08RDE3D00102QZ': [[72    29.150305
   71    29.150334
   70    29.150334
   69    29.150334
   68    29.150047
           ...    
   4     29.150322
   3     29.150322
   2     29.150322
   1     29.150322
   0     29.150322
   Name: Latitude, Length: 73, dtype: float64, 72   -81.048587
   71   -81.048679
   70   -81.048679
   69   -81.048742
   68   -81.048730
           ...    
   4    -81.04807

In [268]:
#for encounter in mapp.mdf.encounterList:
    

    
#     get list of adsb time altitude and coord
    

#     get list of uasEncounters time altitude and coord
#     #get list of adsb time altitude and coord
    
#     compare altitude
#     compare time
#     compare coord
    
#     return list of adsb uas encounterId with time altitude coord for points

encounterPoints = {}


#x = mapp.encounterList.value
for y in mapp.mdf.encounterResults:
    for i in y[:]:
        
        i = i.split(',')[0]
        #if i.split(',')[0] == x:
        print(i.split(',')[0])
        #altitude 
        print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'altitude'][int(i.split(',')[1])][1])
        x1 = mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'altitude'][int(i.split(',')[1])][1]
        #encounterPoints[y] = [mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'altitude'][int(i.split(',')[1])][1],
        #                       mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])],
        #?                        mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'track'][int(i.split(',')[1])]]

        #datetime as string
        print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])])
        x2 = mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])]

        #timerr = mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])]
        #print(timerr)
        #datetime_object = datetime.strptime(timerr, '%Y-%m-%d %H:%M:%S')
        #print( datetime_object + timedelta(hours=6))

        #latitude/longitude
        print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'track'][int(i.split(',')[1])])
        x3 = mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'track'][int(i.split(',')[1])]

        encounterPoints[x] = [x1,x2,x3]
        


        

N8480J


IndexError: list index out of range

In [261]:
x = mapp.encounterList.value
for y in mapp.encounterList.options:
    print(y)

N8480J
N20571
N1230G
N50317
N536ND
N431RP
N714HW


In [204]:
len(mapp.mdf.uasEncounters)

1

In [208]:
mapp.mdf.uasEncounters

[[['N431RP,292'], '163DFAS00180PB', '4']]

In [252]:
type(mapp.encounterList)

ipywidgets.widgets.widget_selection.Dropdown

In [258]:
mapp.encounterList.options

('N8480J', 'N20571', 'N1230G', 'N50317', 'N536ND', 'N431RP', 'N714HW')

In [334]:
for i in mapp.encounterList.options:
    print(i)
    
print(len(mapp.encounterList.options))

N8480J
N20571
N1230G
N50317
N536ND
N431RP
N714HW
7


In [265]:
for z in mapp.encounterList.options:
    
mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'altitude'][int(i.split(',')[1])][1]

N8480J
N20571
N1230G
N50317
N536ND
N431RP
N714HW


In [267]:
for y in mapp.mdf.encounterResults:
    for i in y[:]:
        
        i = i.split(',')[0]
        #if i.split(',')[0] == x:
        print(i.split(',')[0])
        #altitude 
        print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'altitude'][int(i.split(',')[1])][1])
        x1 = mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'altitude'][int(i.split(',')[1])][1]
        #encounterPoints[y] = [mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'altitude'][int(i.split(',')[1])][1],
        #                       mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])],
        #?                        mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'track'][int(i.split(',')[1])]]

        #datetime as string
        print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])])
        x2 = mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])]

        #timerr = mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])]
        #print(timerr)
        #datetime_object = datetime.strptime(timerr, '%Y-%m-%d %H:%M:%S')
        #print( datetime_object + timedelta(hours=6))

        #latitude/longitude
        print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'track'][int(i.split(',')[1])])
        x3 = mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'track'][int(i.split(',')[1])]

        encounterPoints[x] = [x1,x2,x3]


N8480J


IndexError: list index out of range

In [387]:
encounterPoints = {}

for x in mapp.encounterList.options:
    
    for y in mapp.mdf.encounterResults:

        for i in y[:]:

            if i.split(',')[0] == x:


                #print(mapp.mdf.adsbFlightsDictionary[x+'altitude'][int(i.split(',')[1])])
                #print(mapp.mdf.adsbFlightsDictionary[x+'time'][int(i.split(',')[1])])
                #print(mapp.mdf.adsbFlightsDictionary[x+'track'][int(i.split(',')[1])])
                
                
                
                x1 = mapp.mdf.adsbFlightsDictionary[x+'altitude'][int(i.split(',')[1])][1]
        #encounterPoints[y] = [mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'altitude'][int(i.split(',')[1])][1],
        #                       mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])],
        #?                        mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'track'][int(i.split(',')[1])]]

        #datetime as string
        #print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])])
                x2 = mapp.mdf.adsbFlightsDictionary[x+'time'][int(i.split(',')[1])]

#         timerr = mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'time'][int(i.split(',')[1])]
#         print(timerr)
#         datetime_object = datetime.strptime(timerr, '%Y-%m-%d %H:%M:%S')
#         print( datetime_object + timedelta(hours=6))

#         latitude/longitude
#         print(mapp.mdf.adsbFlightsDictionary[mapp.encounterList.value+'track'][int(i.split(',')[1])])
                x3 = mapp.mdf.adsbFlightsDictionary[x+'track'][int(i.split(',')[1])]
            
                try:
                    encounterPoints[x].append([x1,x2,x3])
                except KeyError:
                    encounterPoints[x] = []
                    encounterPoints[x].append([x1,x2,x3])





In [408]:
for y in mapp.mdf.encounterResults:

    for i in y[:]:
        print(i[:])

N536ND,463
N8480J,431
N1230G,450
N1230G,451
N20571,35
N50317,2516
N50317,2517
N536ND,464
N536ND,465
N8480J,425
N8480J,430
N20571,36
N20571,37
N1230G,448
N1230G,449
N8480J,428
N8480J,429
N20571,38
N20571,39
N1230G,446
N1230G,447
N50317,415
N1230G,444
N1230G,445
N8480J,426
N8480J,427
N20571,40
N20571,41
N1230G,442
N1230G,443
N20571,42
N20571,43
N50317,412
N50317,413
N50317,414
N8480J,423
N8480J,424
N1230G,440
N8480J,422
N50317,409
N50317,410
N50317,411
N1230G,441
N20571,44
N20571,45
N20571,46
N536ND,451
N20571,21
N536ND,452
N20571,22
N20571,23
N50317,430
N50317,431
N536ND,453
N20571,24
N20571,25
N50317,428
N50317,429
N50317,425
N50317,426
N50317,427
N20571,26
N50317,422
N50317,423
N50317,424
N1230G,455
N1230G,456
N1230G,457
N20571,15
N50317,436
N536ND,446
N536ND,447
N20571,16
N50317,435
N50317,432
N50317,433
N50317,434
N50317,430
N50317,431
N536ND,443
N536ND,444
N536ND,445
N20571,13
N20571,14
N431RP,291
N431RP,292
N431RP,293
N431RP,294
N431RP,295
N431RP,296
N431RP,297
N536ND,441
N536ND,4

In [344]:
mapp.mdf.adsbFlightsDictionary['N714HW'+'altitude'][2568]

['AirBorn', 975]

In [447]:
for x in mapp.encounterList.options:
    
    for y in mapp.mdf.encounterResults:

        for i in y[:]:

            if i.split(',')[0] == x:

                print(mapp.mdf.adsbFlightsDictionary[x+'altitude'][int(i.split(',')[1])])
                print(mapp.mdf.adsbFlightsDictionary[x+'time'][int(i.split(',')[1])])
                print(mapp.mdf.adsbFlightsDictionary[x+'track'][int(i.split(',')[1])])
                

['AirBorn', 1300]
2019-08-24 14:03:14
[29.15756464004517, -80.97192049026489]
['AirBorn', 1275]
2019-08-24 14:03:15
[29.15780067443848, -80.97202777862549]
['AirBorn', 1275]
2019-08-24 14:03:15
[29.158036708831787, -80.97219944000244]
['AirBorn', 1350]
2019-08-24 14:03:08
[29.154903888702393, -80.97009658813477]
['AirBorn', 1350]
2019-08-24 14:03:08
[29.155139923095703, -80.97031116485596]
['AirBorn', 1350]
2019-08-24 14:03:09
[29.155375957489014, -80.97043991088867]
['AirBorn', 1400]
2019-08-24 14:03:05
[29.15370225906372, -80.9693455696106]
['AirBorn', 1400]
2019-08-24 14:03:05
[29.153916835784912, -80.96945285797119]
['AirBorn', 1425]
2019-08-24 14:03:03
[29.153079986572266, -80.96898078918456]
['AirBorn', 1425]
2019-08-24 14:03:04
[29.153294563293453, -80.96906661987305]
['AirBorn', 1425]
2019-08-24 14:03:04
[29.153423309326172, -80.96917390823364]
['AirBorn', 1450]
2019-08-24 14:03:02
[29.152672290802002, -80.96870183944701]
['AirBorn', 1450]
2019-08-24 14:03:03
[29.15286540985107

In [535]:
encounterPoints = {}

for x in mapp.encounterList.options:
    
    for y in mapp.mdf.encounterResults:

        for i in y[:]:

            if i.split(',')[0] == x:

                x1 = mapp.mdf.adsbFlightsDictionary[x+'altitude'][int(i.split(',')[1])][1]

                x2 = mapp.mdf.adsbFlightsDictionary[x+'time'][int(i.split(',')[1])]


                x3 = mapp.mdf.adsbFlightsDictionary[x+'track'][int(i.split(',')[1])]
            
                try:
                    encounterPoints[x].append([x1,x2,x3])
                except KeyError:
                    encounterPoints[x] = []
                    encounterPoints[x].append([x1,x2,x3])


                    


In [536]:
for i in encounterPoints:
    for z in encounter

{'N536ND': [[1300,
   '2019-08-24 14:03:14',
   [29.15756464004517, -80.97192049026489]],
  [1275, '2019-08-24 14:03:15', [29.15780067443848, -80.97202777862549]],
  [1275, '2019-08-24 14:03:15', [29.158036708831787, -80.97219944000244]],
  [1350, '2019-08-24 14:03:08', [29.154903888702393, -80.97009658813477]],
  [1350, '2019-08-24 14:03:08', [29.155139923095703, -80.97031116485596]],
  [1350, '2019-08-24 14:03:09', [29.155375957489014, -80.97043991088867]],
  [1400, '2019-08-24 14:03:05', [29.15370225906372, -80.9693455696106]],
  [1400, '2019-08-24 14:03:05', [29.153916835784912, -80.96945285797119]],
  [1425, '2019-08-24 14:03:03', [29.153079986572266, -80.96898078918456]],
  [1425, '2019-08-24 14:03:04', [29.153294563293453, -80.96906661987305]],
  [1425, '2019-08-24 14:03:04', [29.153423309326172, -80.96917390823364]],
  [1450, '2019-08-24 14:03:02', [29.152672290802002, -80.96870183944701]],
  [1450, '2019-08-24 14:03:03', [29.152865409851074, -80.96885204315186]],
  [1425, '201

In [512]:
mapp.mdf.uasEncounters[13][1]

'08RDE3V001017Q'

In [506]:
mapp.mdf.uasFlights['08RDE3V001017Q'][0][:]

[9    29.198731
 8    29.198731
 6    29.198708
 7    29.198708
 5    29.198708
 4    29.198708
 3    29.198857
 1    29.199069
 2    29.199069
 0    29.199081
 Name: Latitude, dtype: float64, 9   -80.992443
 8   -80.992443
 6   -80.992695
 7   -80.992718
 5   -80.992724
 4   -80.992724
 3   -80.992546
 1   -80.991750
 2   -80.991750
 0   -80.991710
 Name: Longitude, dtype: float64]

In [665]:
encounterNames = []

for z in mapp.mdf.uasEncounters:
    print(z[1][:14])
    if z[1][:14] in encounterNames:
        ;
    else:
        encounterNames.append(z[1][:14])

print(encounterNames)

for i in encounterNames:
    ;
    
    #print(mapp.mdf.uasOd[i])

163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
['163DFAS00180PB', '08RDE3V001017Q']


In [565]:
encounterNames = []

for z in mapp.mdf.uasEncounters:
    print(z[1][:14])
    try:
        encounterNames[z].append([x1,x2,x3])
    except TypeError:
        encounterNames[z] = []
        encounterNames[z].append([x1,x2,x3])
print(encounterNames)

163DFAS00180PB


TypeError: list indices must be integers or slices, not list

In [566]:
iterator = 0

for uasFlight in mapp.mdf.uasFlights:
    #uasTreeCoordIndex = 0
    for coordinate in mapp.mdf.uasOd[uasFlight][2][iterator]:

        ;
        print(coordinate * 3)
        #print(uasFlight)
    for coordinate in mapp.mdf.uasOd[uasFlight][0][iterator]:
        ;
        #print(coordinate)
        
#         datetime_object = datetime.strptime(coordinate, '%Y-%m-%d %H:%M:%S')
#         print( datetime_object + timedelta(hours=6))
        
#         print(uasFlight)
    for coordinate in mapp.mdf.uasOd[uasFlight][1][iterator]:
        
        ;
        #print(coordinate)
        #print(uasFlight)
    iterator+=1

201.60000000000002
295.79999999999995
355.5
378.29999999999995
378.6
378.29999999999995
377.4
413.09999999999997
413.09999999999997
413.09999999999997
398.40000000000003
406.79999999999995
398.40000000000003
398.40000000000003
398.40000000000003
398.40000000000003
398.40000000000003
374.1
388.5
355.79999999999995
356.1
356.1
355.5
330.9
330.9
330.9
330.9
304.5
291.0
291.0
277.20000000000005
249.89999999999998
250.20000000000002
222.60000000000002
222.60000000000002
185.39999999999998
185.39999999999998
171.89999999999998
162.60000000000002
162.60000000000002
161.39999999999998
161.10000000000002
161.10000000000002
160.8
160.8
160.8
161.10000000000002
161.10000000000002
162.60000000000002
162.60000000000002
162.0
161.7
161.7
162.0
161.7
161.7
161.39999999999998
163.8
163.8
163.2
162.89999999999998
163.2
163.2
163.5
142.2
166.5
166.5
176.10000000000002
178.2
178.2
173.39999999999998
151.8
150.89999999999998
168.3
189.60000000000002
189.60000000000002
240.29999999999998
235.5
212.70000000

In [568]:
encounterNames

[]

In [567]:
for i in encounterNames:
    
    print(i)
    #if len(mapp.mdf.uasOd[i][0]) > 1:
    #    print(mapp.mdf.uasOd[i][0][1])
    

In [530]:
mapp.mdf.uasEncounters

[[['N536ND,463',
   'N8480J,431',
   'N1230G,450',
   'N1230G,451',
   'N20571,35',
   'N50317,2516',
   'N50317,2517',
   'N536ND,464',
   'N536ND,465',
   'N8480J,425',
   'N8480J,430',
   'N20571,36',
   'N20571,37',
   'N1230G,448',
   'N1230G,449',
   'N8480J,428',
   'N8480J,429',
   'N20571,38',
   'N20571,39',
   'N1230G,446',
   'N1230G,447',
   'N50317,415',
   'N1230G,444',
   'N1230G,445',
   'N8480J,426',
   'N8480J,427',
   'N20571,40',
   'N20571,41',
   'N1230G,442',
   'N1230G,443',
   'N20571,42',
   'N20571,43',
   'N50317,412',
   'N50317,413',
   'N50317,414',
   'N8480J,423',
   'N8480J,424',
   'N1230G,440',
   'N8480J,422',
   'N50317,409',
   'N50317,410',
   'N50317,411',
   'N1230G,441',
   'N20571,44',
   'N20571,45',
   'N20571,46'],
  '163DFAS00180PB',
  '0'],
 [['N536ND,451',
   'N20571,21',
   'N536ND,452',
   'N20571,22',
   'N20571,23',
   'N50317,430',
   'N50317,431',
   'N536ND,453',
   'N20571,24',
   'N20571,25',
   'N50317,428',
   'N50317,429',


In [556]:
mapp.mdf.uasFlights['08RDE3V001017Q'][2]

9    46.5
8    49.4
6    49.5
7    49.4
5    49.5
4    49.0
3    33.6
1    52.0
2    52.0
0    50.2
Name: IconAltitude, dtype: float64

In [557]:
encounterPoints[list(encounterPoints.keys())[0]]

[[1300, '2019-08-24 14:03:14', [29.15756464004517, -80.97192049026489]],
 [1275, '2019-08-24 14:03:15', [29.15780067443848, -80.97202777862549]],
 [1275, '2019-08-24 14:03:15', [29.158036708831787, -80.97219944000244]],
 [1350, '2019-08-24 14:03:08', [29.154903888702393, -80.97009658813477]],
 [1350, '2019-08-24 14:03:08', [29.155139923095703, -80.97031116485596]],
 [1350, '2019-08-24 14:03:09', [29.155375957489014, -80.97043991088867]],
 [1400, '2019-08-24 14:03:05', [29.15370225906372, -80.9693455696106]],
 [1400, '2019-08-24 14:03:05', [29.153916835784912, -80.96945285797119]],
 [1425, '2019-08-24 14:03:03', [29.153079986572266, -80.96898078918456]],
 [1425, '2019-08-24 14:03:04', [29.153294563293453, -80.96906661987305]],
 [1425, '2019-08-24 14:03:04', [29.153423309326172, -80.96917390823364]],
 [1450, '2019-08-24 14:03:02', [29.152672290802002, -80.96870183944701]],
 [1450, '2019-08-24 14:03:03', [29.152865409851074, -80.96885204315186]],
 [1425, '2019-08-24 14:03:03', [29.1530799

In [ ]:
for i in encounterNames:
    
    for mapp.mdf.uasFlights[i][2]
    
        for z in encounterPoints:
            
            if encounterPoints[z]
    
    

In [580]:
for adsb in encounterPoints[list(encounterPoints.keys())[0]]:
    
    
    
    print(adsb)#[2])

[1300, '2019-08-24 14:03:14', [29.15756464004517, -80.97192049026489]]
[1275, '2019-08-24 14:03:15', [29.15780067443848, -80.97202777862549]]
[1275, '2019-08-24 14:03:15', [29.158036708831787, -80.97219944000244]]
[1350, '2019-08-24 14:03:08', [29.154903888702393, -80.97009658813477]]
[1350, '2019-08-24 14:03:08', [29.155139923095703, -80.97031116485596]]
[1350, '2019-08-24 14:03:09', [29.155375957489014, -80.97043991088867]]
[1400, '2019-08-24 14:03:05', [29.15370225906372, -80.9693455696106]]
[1400, '2019-08-24 14:03:05', [29.153916835784912, -80.96945285797119]]
[1425, '2019-08-24 14:03:03', [29.153079986572266, -80.96898078918456]]
[1425, '2019-08-24 14:03:04', [29.153294563293453, -80.96906661987305]]
[1425, '2019-08-24 14:03:04', [29.153423309326172, -80.96917390823364]]
[1450, '2019-08-24 14:03:02', [29.152672290802002, -80.96870183944701]]
[1450, '2019-08-24 14:03:03', [29.152865409851074, -80.96885204315186]]
[1425, '2019-08-24 14:03:03', [29.153079986572266, -80.9689807891845

In [564]:
for uas in mapp.mdf.uasEncounters:
    print(uas[1])

163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB1
163DFAS00180PB1
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q


In [585]:
if'163DFAS00180PB' in '163DFAS00180PB11':
    print('r')

r


In [586]:
newlist

[['N536ND,438',
  'N20571,7',
  'N20571,8',
  'N431RP,287',
  'N431RP,288',
  'N536ND,439',
  'N536ND,440',
  'N536ND,441',
  'N536ND,442',
  'N20571,9',
  'N20571,10',
  'N20571,11',
  'N431RP,289',
  'N431RP,290',
  'N536ND,443',
  'N536ND,444',
  'N20571,12',
  'N20571,13',
  'N431RP,291',
  'N431RP,292'],
 '163DFAS00180PB',
 '8']

In [884]:
import datetime

# list(mapp.mdf.uasEncounters[0])
# mapp.mdf.uasFlights[]

form = '%Y-%m-%d %H:%M:%S'

#minnnn = min(mapp.mdf.uasFlights[i[1][:14]][2])
#minnnn

p = []


for i in list(mapp.mdf.uasEncounters[:]):
    
    #altLow = min(mapp.mdf.uasFlights[i[1][:14]][2]) * 3.28084
    #lo = i[1]
    
    timeindex = int(i[1][14:] + i[2])
    #print(timeindex)
    
    
    timeLow = mapp.mdf.uasFlights[i[1][:14]][1][timeindex]
    print('uas')
    print(timeLow)

    for z in i[:][0]:
        print('adsb')
        print(mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'time']
              [int(z.split(',')[1])])

        timeA = datetime.datetime.strptime(timeLow,form )
        timeB = datetime.datetime.strptime(mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'time']
              [int(z.split(',')[1])], form)
        
        if timeA>timeB:
            diff = timeA-timeB
        else:
            
            diff = (timeB - timeA)
        print(diff)
        
        if diff > timedelta(minutes = 60):
            i[0].remove(z)
            print('removed', z , "from" , i[1][:14] + i[2])
            
        
    
            
        
        
#         print(time.strptime(timeLow,form ) - time.strptime(mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'time']
#               [int(z.split(',')[1])], form))
    #altHigh = max(mapp.mdf.uasFlights[i[1][:14]][2]) * 3.28084
#     print(i[1][:14])
#     print(i[1]+i[2])
#     #print(i[0])
#     print(altLow, altHigh)
#     for z in i[:][0]:

form = '%y/%m/%d %H:%M:%S'



uas
2019-08-24 17:02:18
adsb
2019-08-24 21:58:18
4:56:00
removed N62568,2713 from 08RDEA200101WW0
adsb
2019-08-24 21:58:19
4:56:01
removed N62568,2715 from 08RDEA200101WW0
adsb
2019-08-24 21:58:20
4:56:02
removed N62568,2717 from 08RDEA200101WW0
adsb
2019-08-24 21:58:17
4:55:59
removed N62568,2711 from 08RDEA200101WW0
adsb
2019-08-24 13:52:07
3:10:11
removed N5144H,1304 from 08RDEA200101WW0
adsb
2019-08-24 13:52:08
3:10:10
removed N5144H,1306 from 08RDEA200101WW0
adsb
2019-08-24 13:52:09
3:10:09
removed N5144H,1308 from 08RDEA200101WW0
adsb
2019-08-24 13:52:10
3:10:08
removed N5144H,1310 from 08RDEA200101WW0
adsb
2019-08-24 19:06:21
2:04:03
removed N882TC,6992 from 08RDEA200101WW0
adsb
2019-08-24 19:06:18
2:04:00
removed N882TC,6986 from 08RDEA200101WW0
adsb
2019-08-24 19:06:19
2:04:01
removed N882TC,6988 from 08RDEA200101WW0
adsb
2019-08-24 19:06:20
2:04:02
removed N882TC,6990 from 08RDEA200101WW0
adsb
2019-08-24 19:06:17
2:03:59
removed N882TC,6984 from 08RDEA200101WW0
adsb
2019-08-2

removed N7YJ,1191 from 08RDEA200101WW0
adsb
2019-08-24 00:11:02
16:51:16
removed N7YJ,1193 from 08RDEA200101WW0
adsb
2019-08-24 00:11:03
16:51:15
removed N7YJ,1195 from 08RDEA200101WW0
adsb
2019-08-24 00:08:23
16:53:55
removed N7YJ,898 from 08RDEA200101WW0
adsb
2019-08-24 00:08:24
16:53:54
removed N7YJ,900 from 08RDEA200101WW0
adsb
2019-08-24 00:08:25
16:53:53
removed N7YJ,902 from 08RDEA200101WW0
adsb
2019-08-24 00:08:26
16:53:52
removed N7YJ,904 from 08RDEA200101WW0
adsb
2019-08-24 00:08:27
16:53:51
removed N7YJ,906 from 08RDEA200101WW0
adsb
2019-08-24 00:08:28
16:53:50
removed N7YJ,908 from 08RDEA200101WW0
adsb
2019-08-24 00:08:29
16:53:49
removed N7YJ,910 from 08RDEA200101WW0
adsb
2019-08-24 00:08:30
16:53:48
removed N7YJ,912 from 08RDEA200101WW0
adsb
2019-08-24 00:08:31
16:53:47
removed N7YJ,914 from 08RDEA200101WW0
adsb
2019-08-24 00:08:32
16:53:46
removed N7YJ,916 from 08RDEA200101WW0
adsb
2019-08-24 00:08:33
16:53:45
removed N7YJ,918 from 08RDEA200101WW0
adsb
2019-08-24 00:08:3

removed N7YJ,1600 from 08RDEA200101WW0
adsb
2019-08-24 00:14:41
16:47:37
removed N7YJ,1602 from 08RDEA200101WW0
adsb
2019-08-24 00:14:42
16:47:36
removed N7YJ,1604 from 08RDEA200101WW0
adsb
2019-08-24 00:14:43
16:47:35
removed N7YJ,1606 from 08RDEA200101WW0
adsb
2019-08-24 00:14:44
16:47:34
removed N7YJ,1608 from 08RDEA200101WW0
adsb
2019-08-24 00:14:45
16:47:33
removed N7YJ,1610 from 08RDEA200101WW0
adsb
2019-08-24 00:14:46
16:47:32
removed N7YJ,1612 from 08RDEA200101WW0
adsb
2019-08-24 00:14:49
16:47:29
removed N7YJ,1614 from 08RDEA200101WW0
adsb
2019-08-24 00:14:50
16:47:28
removed N7YJ,1616 from 08RDEA200101WW0
adsb
2019-08-24 00:15:02
16:47:16
removed N7YJ,1638 from 08RDEA200101WW0
adsb
2019-08-24 00:15:03
16:47:15
removed N7YJ,1640 from 08RDEA200101WW0
adsb
2019-08-24 00:15:04
16:47:14
removed N7YJ,1642 from 08RDEA200101WW0
adsb
2019-08-24 00:03:29
16:58:49
removed N7YJ,365 from 08RDEA200101WW0
adsb
2019-08-24 00:03:30
16:58:48
removed N7YJ,367 from 08RDEA200101WW0
adsb
2019-08-2

2019-08-24 00:03:57
16:58:21
removed N7YJ,417 from 08RDEA200101WW0
adsb
2019-08-24 00:03:58
16:58:20
removed N7YJ,419 from 08RDEA200101WW0
adsb
2019-08-24 00:03:59
16:58:19
removed N7YJ,421 from 08RDEA200101WW0
adsb
2019-08-24 00:04:00
16:58:18
removed N7YJ,423 from 08RDEA200101WW0
adsb
2019-08-24 00:04:01
16:58:17
removed N7YJ,425 from 08RDEA200101WW0
adsb
2019-08-24 00:16:53
16:45:25
removed N7YJ,1830 from 08RDEA200101WW0
adsb
2019-08-24 00:16:54
16:45:24
removed N7YJ,1832 from 08RDEA200101WW0
adsb
2019-08-24 00:04:02
16:58:16
removed N7YJ,426 from 08RDEA200101WW0
adsb
2019-08-24 00:04:03
16:58:15
removed N7YJ,428 from 08RDEA200101WW0
adsb
2019-08-24 00:04:04
16:58:14
removed N7YJ,430 from 08RDEA200101WW0
adsb
2019-08-24 00:04:04
16:58:14
removed N7YJ,432 from 08RDEA200101WW0
adsb
2019-08-24 00:04:05
16:58:13
removed N7YJ,434 from 08RDEA200101WW0
adsb
2019-08-24 00:04:07
16:58:11
removed N7YJ,436 from 08RDEA200101WW0
adsb
2019-08-24 00:04:08
16:58:10
removed N7YJ,438 from 08RDEA20010

removed N7YJ,1889 from 08RDEA200101WW0
adsb
2019-08-24 13:54:34
3:07:44
removed N524ND,1059 from 08RDEA200101WW0
adsb
2019-08-24 13:54:35
3:07:43
removed N524ND,1061 from 08RDEA200101WW0
adsb
2019-08-24 00:17:28
16:44:50
removed N7YJ,1892 from 08RDEA200101WW0
adsb
2019-08-24 13:54:36
3:07:42
removed N524ND,1062 from 08RDEA200101WW0
adsb
2019-08-24 00:17:29
16:44:49
removed N7YJ,1894 from 08RDEA200101WW0
adsb
2019-08-24 00:06:47
16:55:31
removed N7YJ,728 from 08RDEA200101WW0
adsb
2019-08-24 00:06:48
16:55:30
removed N7YJ,730 from 08RDEA200101WW0
adsb
2019-08-24 00:06:50
16:55:28
removed N7YJ,734 from 08RDEA200101WW0
adsb
2019-08-24 00:06:51
16:55:27
removed N7YJ,736 from 08RDEA200101WW0
adsb
2019-08-24 00:06:52
16:55:26
removed N7YJ,738 from 08RDEA200101WW0
adsb
2019-08-24 00:06:53
16:55:25
removed N7YJ,740 from 08RDEA200101WW0
adsb
2019-08-24 00:06:54
16:55:24
removed N7YJ,742 from 08RDEA200101WW0
adsb
2019-08-24 00:06:41
16:55:37
removed N7YJ,716 from 08RDEA200101WW0
adsb
2019-08-24 0

...

4:45:48
removed N143CC,3029 from 08RDE3V001017Q9
adsb
2019-08-24 15:59:05
4:45:47
removed N143CC,3031 from 08RDE3V001017Q9
adsb
2019-08-24 15:59:07
4:45:45
removed N143CC,3033 from 08RDE3V001017Q9
adsb
2019-08-24 15:29:03
5:15:49
removed N143CC,1359 from 08RDE3V001017Q9
adsb
2019-08-24 15:29:04
5:15:48
removed N143CC,1361 from 08RDE3V001017Q9
adsb
2019-08-24 17:37:06
3:07:46
removed N20571,289 from 08RDE3V001017Q9
adsb
2019-08-24 17:37:07
3:07:45
removed N20571,291 from 08RDE3V001017Q9
adsb
2019-08-24 15:29:02
5:15:50
removed N143CC,1356 from 08RDE3V001017Q9
adsb
2019-08-24 17:37:07
3:07:45
removed N20571,290 from 08RDE3V001017Q9
adsb
2019-08-24 17:54:03
2:50:49
removed N2504M,5430 from 08RDE3V001017Q9
adsb
2019-08-24 16:19:34
4:25:18
removed N143CC,4467 from 08RDE3V001017Q9
adsb
2019-08-24 16:19:35
4:25:17
removed N143CC,4469 from 08RDE3V001017Q9
adsb
2019-08-24 16:14:28
4:30:24
removed N2504M,2602 from 08RDE3V001017Q9
adsb
2019-08-24 18:23:06
2:21:46
removed N2504M,7540 from 08RDE3V0

removed RDDL942,5276 from 163DG6F00175B10
adsb
2019-08-24 22:46:07
1:39:10
removed N5144H,5202 from 163DG6F00175B10
adsb
2019-08-24 16:59:35
4:07:22
removed RDDL475,535 from 163DG6F00175B10
adsb
2019-08-24 16:32:46
4:34:11
removed N1448Z,1421 from 163DG6F00175B10
adsb
2019-08-24 00:26:42
20:40:15
removed N52246,97 from 163DG6F00175B10
adsb
2019-08-24 16:59:36
4:07:21
removed RDDL475,537 from 163DG6F00175B10
adsb
2019-08-24 00:39:34
20:27:23
removed N50317,1644 from 163DG6F00175B10
adsb
2019-08-24 00:26:41
20:40:16
removed N52246,94 from 163DG6F00175B10
adsb
2019-08-24 00:26:42
20:40:15
removed N52246,96 from 163DG6F00175B10
adsb
2019-08-24 23:45:08
2:38:11
removed N1159K,2087 from 163DG6F00175B10
adsb
2019-08-24 23:13:47
2:06:50
removed RDDL942,5275 from 163DG6F00175B10
adsb
2019-08-24 14:40:33
6:26:24
removed N536ND,1146 from 163DG6F00175B10
adsb
2019-08-24 14:18:51
6:48:06
removed RDDL491,1665 from 163DG6F00175B10
adsb
2019-08-24 14:40:32
6:26:25
removed N536ND,1145 from 163DG6F00175

removed N5144H,4172 from 163DG6F00175B10
adsb
2019-08-24 23:45:20
2:38:23
removed N1159K,2112 from 163DG6F00175B10
adsb
2019-08-24 16:33:00
4:33:57
removed N1448Z,1442 from 163DG6F00175B10
adsb
2019-08-24 14:19:05
6:47:52
removed RDDL491,1689 from 163DG6F00175B10
adsb
2019-08-24 16:59:47
4:07:10
removed RDDL475,559 from 163DG6F00175B10
adsb
2019-08-24 01:27:40
19:39:17
removed RDDL362,1422 from 163DG6F00175B10
adsb
2019-08-24 00:26:54
20:40:03
removed N52246,119 from 163DG6F00175B10
adsb
2019-08-24 16:59:47
4:07:10
removed RDDL475,558 from 163DG6F00175B10
adsb
2019-08-24 22:46:21
1:39:24
removed N5144H,5228 from 163DG6F00175B10
adsb
2019-08-24 00:39:49
20:27:08
removed N50317,1670 from 163DG6F00175B10
adsb
2019-08-24 19:42:21
1:24:36
removed N5144H,4171 from 163DG6F00175B10
adsb
2019-08-24 16:32:59
4:33:58
removed N1448Z,1441 from 163DG6F00175B10
adsb
2019-08-24 20:24:08
0:42:49
adsb
2019-08-24 14:21:31
6:45:26
removed RDDL414,2251 from 163DG6F00175B10
adsb
2019-08-24 16:32:59
4:33:58


2019-08-24 01:27:30
19:38:25
removed RDDL362,1401 from 163DG6F00175B11
adsb
2019-08-24 20:23:58
0:41:57
adsb
2019-08-24 23:13:49
2:07:54
removed RDDL942,5280 from 163DG6F00175B11
adsb
2019-08-24 14:40:35
6:25:20
removed N536ND,1150 from 163DG6F00175B11
adsb
2019-08-24 20:23:58
0:41:57
adsb
2019-08-24 14:18:55
6:47:00
removed RDDL491,1672 from 163DG6F00175B11
adsb
2019-08-24 22:46:10
1:40:15
removed N5144H,5207 from 163DG6F00175B11
adsb
2019-08-24 12:29:58
8:35:57
removed N559ND,786 from 163DG6F00175B11
adsb
2019-08-24 12:29:59
8:35:56
removed N559ND,788 from 163DG6F00175B11
adsb
2019-08-24 22:46:09
1:40:14
removed N5144H,5205 from 163DG6F00175B11
adsb
2019-08-24 16:59:37
4:06:18
removed RDDL475,538 from 163DG6F00175B11
adsb
2019-08-24 00:39:37
20:26:18
removed N50317,1648 from 163DG6F00175B11
adsb
2019-08-24 21:59:18
0:53:23
adsb
2019-08-24 21:59:15
0:53:20
adsb
2019-08-24 19:30:41
1:35:14
removed N52246,3595 from 163DG6F00175B11
adsb
2019-08-24 19:42:07
1:23:48
removed N5144H,4145 fro

removed N52246,3630 from 163DG6F00175B11
adsb
2019-08-24 21:59:34
0:53:39
adsb
2019-08-24 21:59:34
0:53:39
adsb
2019-08-24 16:26:33
4:39:22
removed RDDL942,1283 from 163DG6F00175B11
adsb
2019-08-24 20:24:12
0:41:43
adsb
2019-08-24 19:42:25
1:23:30
removed N5144H,4179 from 163DG6F00175B11
adsb
2019-08-24 14:21:34
6:44:21
removed RDDL414,2257 from 163DG6F00175B11
adsb
2019-08-24 23:45:24
2:39:29
removed N1159K,2118 from 163DG6F00175B11
adsb
2019-08-24 16:33:04
4:32:51
removed N1448Z,1449 from 163DG6F00175B11
adsb
2019-08-24 14:19:08
6:46:47
removed RDDL491,1696 from 163DG6F00175B11
adsb
2019-08-24 16:59:51
4:06:04
removed RDDL475,566 from 163DG6F00175B11
adsb
2019-08-24 14:40:51
6:25:04
removed N536ND,1180 from 163DG6F00175B11
adsb
2019-08-24 22:46:26
1:40:31
removed N5144H,5236 from 163DG6F00175B11
adsb
2019-08-24 23:14:04
2:08:09
removed RDDL942,5306 from 163DG6F00175B11
adsb
2019-08-24 14:40:52
6:25:03
removed N536ND,1179 from 163DG6F00175B11
adsb
2019-08-24 01:27:44
19:38:11
removed 

altitude check below, removes elements from list that are without the range

time check is next



In [922]:
p = []

for i in list(mapp.mdf.uasEncounters[:]):
    
    altLow = min(mapp.mdf.uasFlights[i[1][:14]][2]) * 3.28084
    altHigh = max(mapp.mdf.uasFlights[i[1][:14]][2]) * 3.28084
    print(i[1][:14])
    print(i[1]+i[2])
    #print(i[0])
    print(altLow, altHigh)
    for z in i[:][0]:
        
        #pmax = max(mapp.mdf.uasFlights[i][2][:])
        #pmin = min(mapp.mdf.uasFlights[i][2][:])
        
        if (mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'altitude']
        [int(z.split(',')[1])]
        [1]) >altLow and (mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'altitude']
        [int(z.split(',')[1])]
        [1]) < altHigh:
            print(mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'altitude']
                  [int(z.split(',')[1])])
            print('pass')
            
            #print(mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'altitude']
#[int(z.split(',')[1])]
#[1])
            #print(altLow)
            #print(altHigh)
        else:
            print(mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'altitude']
                  [int(z.split(',')[1])])
            i[0].remove(z)
            print('removed', z , "from" , i[1][:14] + i[2])
            print(' ')
            #i[0].remove(z)
            
    
    p.append(i)
print(p)
#print(altLow, altHigh)

print(mapp.mdf.uasEncounters)

163DFAS00180PB
163DFAS00180PB0
223.09712 250.65617600000002
163DFAS00180PB
163DFAS00180PB1
223.09712 250.65617600000002
163DFAS00180PB
163DFAS00180PB3
223.09712 250.65617600000002
163DFAS00180PB
163DFAS00180PB4
223.09712 250.65617600000002
163DFAS00180PB
163DFAS00180PB8
223.09712 250.65617600000002
163DFAS00180PB
163DFAS00180PB9
223.09712 250.65617600000002
163DFAS00180PB
163DFAS00180PB10
223.09712 250.65617600000002
163DFAS00180PB
163DFAS00180PB11
223.09712 250.65617600000002
08RDE3V001017Q
08RDE3V001017Q3
110.236224 170.60368
08RDE3V001017Q
08RDE3V001017Q4
110.236224 170.60368
08RDE3V001017Q
08RDE3V001017Q5
110.236224 170.60368
08RDE3V001017Q
08RDE3V001017Q6
110.236224 170.60368
08RDE3V001017Q
08RDE3V001017Q7
110.236224 170.60368
08RDE3V001017Q
08RDE3V001017Q8
110.236224 170.60368
08RDE3V001017Q
08RDE3V001017Q9
110.236224 170.60368
[[[], '163DFAS00180PB', '0'], [[], '163DFAS00180PB', '1'], [[], '163DFAS00180PB', '3'], [[], '163DFAS00180PB', '4'], [[], '163DFAS00180PB', '8'], [[], '16

In [893]:
for i in p:
    print(len(i[0]) , i[1] + i[2])

528 08RDEA200101WW0
809 163DFAS00180PB0
626 163DFAS00180PB1
505 163DFAS00180PB2
470 163DFAS00180PB3
460 163DFAS00180PB4
429 163DFAS00180PB5
406 163DFAS00180PB6
406 163DFAS00180PB7
426 163DFAS00180PB8
450 163DFAS00180PB9
488 163DFAS00180PB10
631 163DFAS00180PB11
12 07JDE3300201B00
8 07JDE3300201B01
8 07JDE3300201B02
6 07JDE3300201B03
7 07JDE3300201B04
6 07JDE3300201B05
5 07JDE3300201B06
5 07JDE3300201B07
5 07JDE3300201B08
5 07JDE3300201B09
5 07JDE3300201B010
5 07JDE3300201B011
5 07JDE3300201B012
5 07JDE3300201B013
6 07JDE3300201B014
5 07JDE3300201B015
8 07JDE3300201B016
8 07JDE3300201B017
10 07JDE3300201B018
10 07JDE3300201B019
11 07JDE3300201B020
5 B7JDE3300201B00
6 B7JDE3300201B01
567 08RDE3V001017Q0
567 08RDE3V001017Q1
567 08RDE3V001017Q2
550 08RDE3V001017Q3
545 08RDE3V001017Q4
545 08RDE3V001017Q5
544 08RDE3V001017Q6
545 08RDE3V001017Q7
550 08RDE3V001017Q8
550 08RDE3V001017Q9
269 163DG6F00175B10
273 163DG6F00175B11


In [ ]:
#         print(mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'altitude']
# [int(z.split(',')[1])]
# [1])
    #print(i[0][0])
        #print(z)
        #print(z[0].split(',')[0])
        #for i in mapp.mdf.uasFlights['08RDE3V001017Q'][2][:]
        
        
        #print(mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'altitude']
        #[int(z.split(',')[1])]
        #[1])

#         for p in encounterNames:
            
#             for u in mapp.mdf.uasFlights[p][2][:]:

#         if (u*3 <  mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'altitude']
# [int(z.split(',')[1])]
# [1]):
#             print(u*3)

        #print(adsbAltitude)
        #print()
        
    #print(i[1]+i[2])
    
    #mapp.mdf.adsbFlightsDictionary['N714HW'+'altitude'][2568]
    
    
    
    #print(i[0][0])

In [815]:
for i in list(mapp.mdf.uasEncounters[:]):
    
    altLow = min(mapp.mdf.uasFlights[i[1][:14]][2]) * 3 + 400
    altHigh = max(mapp.mdf.uasFlights[i[1][:14]][2]) * 3 + 2000
    #print(i[1][:14])
    for z in i[:][0]:
        print(z)
        print(i)
        print(i[0].remove(z))
        print(i)

# for i in list(mapp.mdf.uasEncounters[:]):
#     print(i[1][:14])

N8480J,425
[['N8480J,425', 'N8480J,430', 'N20571,37', 'N1230G,448', 'N1230G,449', 'N8480J,428', 'N8480J,429', 'N20571,38', 'N20571,39', 'N1230G,446', 'N1230G,447', 'N1230G,444', 'N1230G,445', 'N8480J,426', 'N8480J,427', 'N20571,40', 'N20571,41', 'N1230G,442', 'N1230G,443', 'N20571,42', 'N20571,43', 'N50317,412', 'N50317,410', 'N50317,411', 'N1230G,441', 'N20571,44', 'N20571,45'], '163DFAS00180PB', '0']
None
[['N8480J,430', 'N20571,37', 'N1230G,448', 'N1230G,449', 'N8480J,428', 'N8480J,429', 'N20571,38', 'N20571,39', 'N1230G,446', 'N1230G,447', 'N1230G,444', 'N1230G,445', 'N8480J,426', 'N8480J,427', 'N20571,40', 'N20571,41', 'N1230G,442', 'N1230G,443', 'N20571,42', 'N20571,43', 'N50317,412', 'N50317,410', 'N50317,411', 'N1230G,441', 'N20571,44', 'N20571,45'], '163DFAS00180PB', '0']
N20571,37
[['N8480J,430', 'N20571,37', 'N1230G,448', 'N1230G,449', 'N8480J,428', 'N8480J,429', 'N20571,38', 'N20571,39', 'N1230G,446', 'N1230G,447', 'N1230G,444', 'N1230G,445', 'N8480J,426', 'N8480J,427', 'N2

In [705]:

#for z in mapp.mdf.uas

for p in encounterNames:
    for i in mapp.mdf.uasFlights[p][2][:]:
        if i * 3 < 400:
            print(i * 3)
# for i in list(mapp.mdf.uasEncounters[:]):
#     print(i[0][0])

129.89999999999998
204.0
229.20000000000002
228.60000000000002
229.20000000000002
227.39999999999998
227.39999999999998
226.79999999999998
227.10000000000002
227.10000000000002
227.70000000000002
227.70000000000002
229.20000000000002
228.29999999999998
227.70000000000002
227.70000000000002
228.0
228.0
227.70000000000002
227.70000000000002
227.70000000000002
227.39999999999998
227.70000000000002
228.0
228.0
228.29999999999998
228.60000000000002
228.60000000000002
229.20000000000002
240.60000000000002
230.10000000000002
297.6
327.9
251.70000000000002
227.70000000000002
228.60000000000002
139.5
148.2
148.5
148.2
148.5
147.0
100.80000000000001
156.0
156.0
150.60000000000002
150.0
178.8


In [625]:
encounterNames

[]

In [638]:
mapp.mdf.uasFlights.keys()

dict_keys(['08RDE620010192', '08RDE3D00102QZ', '163DG1A001DPZJ', '08RDE6Q00102N9', '0K1CF5S3AE1EMT', '07JDEB600201L6', '163DF810013D1X', '163DFBM001PRNP', '163DFAC001ZQP0', '08RDE8G00104KD', '08RDEB600103TS', '11VKF2P00200S5', '08RDE1H00102Q0', '08RDEB400103ZM', '0M6DF6N00198G8', '0BMUFAB00100AF', '163DFBM001629Q', '08RDDC5001023K', '08RDE6100104DH', '0K1UF6600F0070', '163DF8J0011M92', '11VKF6P00202K6', '08RDE3T001002Z', '07JDDAJ00100S6', '163DG1Q00152MJ', '08RDEA300103T0', '08RDE8X001015W', '163DF8L001MME2', '03Z0017922', '08RDE8700100YS', '163DG1Q001LMYM', '08RDE8F001021K', '03Z1194928', '163DF8M0013L99', '03Z1072331', '163DFCP001N94W', '08RDDBF00100Q2', '0M6DF91001678J', '163DFA6001VBKR', '0BMUE8M00101P7', '163DFAB001C56H', '08RDE1700104WP', 'nan', '276DFAF0017X1Z', '0BMLE7F30100VS', '0K1DEC92BC0033', '0M63G2R00SH037', '07JDE4R00A00DH', '08RDE7C00103LX', '0BMLE5301501QA', '08RDDCT00103YA', '08RDE7A00103S4', '08RDEA300101K6', '03Z0504514', '07JDD8P001001S', '11VKF7M00200G8', '163DF7X

In [662]:
o=0
encounterNameList = []

for i in mapp.mdf.uasEncounters:
    
    
    #x = i[-2:][:14]
    x = i[-2:][:14]
    x = x[0][:14]
    
    #y = i[14:]
    #x = i[-2:][0][:14].split(',')
    #print(i[-2:][0][:14])
    #print(i[-2:][1])
    #o+=1
    print(x)
    if x in encounterNameList:
        ;
    else:
        encounterNameList.append(x)
    
    

163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q


In [663]:
encounterNameList

['163DFAS00180PB', '08RDE3V001017Q']

In [674]:
encounterNames = []

for z in mapp.mdf.uasEncounters:
    print(z[1][:14])
    if z[1][:14] in encounterNames:
        ;
    else:
        encounterNames.append(z[1][:14])

print(encounterNames)



08RDEA200101WW
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
163DFAS00180PB
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
07JDE3300201B0
B7JDE3300201B0
B7JDE3300201B0
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
08RDE3V001017Q
163DG6F00175B1
163DG6F00175B1
['08RDEA200101WW', '163DFAS00180PB', '07JDE3300201B0', 'B7JDE3300201B0', '08RDE3V001017Q', '163DG6F00175B1']


In [715]:
mapp.mdf.uasEncounters

[[['N8480J,425',
   'N8480J,430',
   'N20571,37',
   'N1230G,448',
   'N1230G,449',
   'N8480J,428',
   'N8480J,429',
   'N20571,38',
   'N20571,39',
   'N1230G,446',
   'N1230G,447',
   'N1230G,444',
   'N1230G,445',
   'N8480J,426',
   'N8480J,427',
   'N20571,40',
   'N20571,41',
   'N1230G,442',
   'N1230G,443',
   'N20571,42',
   'N20571,43',
   'N50317,412',
   'N50317,410',
   'N50317,411',
   'N1230G,441',
   'N20571,44',
   'N20571,45'],
  '163DFAS00180PB',
  '0'],
 [['N20571,23',
   'N50317,430',
   'N20571,24',
   'N50317,428',
   'N50317,429',
   'N50317,425',
   'N50317,426',
   'N50317,427',
   'N50317,423',
   'N50317,424',
   'N1230G,457'],
  '163DFAS00180PB',
  '1'],
 [['N536ND,445',
   'N20571,14',
   'N431RP,292',
   'N431RP,293',
   'N431RP,294',
   'N431RP,295',
   'N431RP,296',
   'N431RP,297'],
  '163DFAS00180PB',
  '2'],
 [['N536ND,442',
   'N431RP,291',
   'N431RP,292',
   'N431RP,293',
   'N431RP,294',
   'N431RP,295'],
  '163DFAS00180PB',
  '3'],
 [['N536ND,4

In [730]:
popp = mapp.mdf.uasEncounters

In [733]:
popp[0][0].remove(popp[0][0][0])

In [735]:
popp

[[['N8480J,428',
   'N8480J,429',
   'N20571,38',
   'N20571,39',
   'N1230G,446',
   'N1230G,447',
   'N1230G,444',
   'N1230G,445',
   'N8480J,426',
   'N8480J,427',
   'N20571,40',
   'N20571,41',
   'N1230G,442',
   'N1230G,443',
   'N20571,42',
   'N20571,43',
   'N50317,412',
   'N50317,410',
   'N50317,411',
   'N1230G,441',
   'N20571,44',
   'N20571,45'],
  '163DFAS00180PB',
  '0'],
 [['N20571,23',
   'N50317,430',
   'N20571,24',
   'N50317,428',
   'N50317,429',
   'N50317,425',
   'N50317,426',
   'N50317,427',
   'N50317,423',
   'N50317,424',
   'N1230G,457'],
  '163DFAS00180PB',
  '1'],
 [['N536ND,445',
   'N20571,14',
   'N431RP,292',
   'N431RP,293',
   'N431RP,294',
   'N431RP,295',
   'N431RP,296',
   'N431RP,297'],
  '163DFAS00180PB',
  '2'],
 [['N536ND,442',
   'N431RP,291',
   'N431RP,292',
   'N431RP,293',
   'N431RP,294',
   'N431RP,295'],
  '163DFAS00180PB',
  '3'],
 [['N536ND,441',
   'N431RP,290',
   'N431RP,291',
   'N431RP,292',
   'N431RP,293',
   'N431RP,

In [763]:

#for z in mapp.mdf.uas

for p in encounterNames:
    for i in mapp.mdf.uasFlights[p][2][:]:
        
        print(i)
        print(p)

43.3
08RDEA200101WW
68.0
163DFAS00180PB
76.4
163DFAS00180PB
76.2
163DFAS00180PB
76.4
163DFAS00180PB
75.8
163DFAS00180PB
75.8
163DFAS00180PB
75.6
163DFAS00180PB
75.7
163DFAS00180PB
75.7
163DFAS00180PB
75.9
163DFAS00180PB
75.9
163DFAS00180PB
76.4
163DFAS00180PB
76.1
07JDE3300201B0
75.9
07JDE3300201B0
75.9
07JDE3300201B0
76.0
07JDE3300201B0
76.0
07JDE3300201B0
75.9
07JDE3300201B0
75.9
07JDE3300201B0
75.9
07JDE3300201B0
75.8
07JDE3300201B0
75.9
07JDE3300201B0
76.0
07JDE3300201B0
76.0
07JDE3300201B0
76.1
07JDE3300201B0
76.2
07JDE3300201B0
76.2
07JDE3300201B0
76.4
07JDE3300201B0
80.2
07JDE3300201B0
76.7
07JDE3300201B0
99.2
07JDE3300201B0
109.3
07JDE3300201B0
83.9
07JDE3300201B0
75.9
B7JDE3300201B0
76.2
B7JDE3300201B0
46.5
08RDE3V001017Q
49.4
08RDE3V001017Q
49.5
08RDE3V001017Q
49.4
08RDE3V001017Q
49.5
08RDE3V001017Q
49.0
08RDE3V001017Q
33.6
08RDE3V001017Q
52.0
08RDE3V001017Q
52.0
08RDE3V001017Q
50.2
08RDE3V001017Q
50.0
163DG6F00175B1
59.6
163DG6F00175B1


In [738]:
mapp.mdf.uasFlights

{'08RDE620010192': [[5    29.194188
   4    29.194188
   3    29.194371
   2    29.194446
   1    29.194463
   0    29.194463
   Name: Latitude, dtype: float64, 5   -81.047968
   4   -81.047963
   3   -81.047516
   2   -81.047384
   1   -81.047332
   0   -81.047332
   Name: Longitude, dtype: float64], 5    2019-08-14 15:21:14
  4    2019-08-14 15:21:40
  3    2019-08-14 15:23:12
  2    2019-08-14 15:23:26
  1    2019-08-14 15:25:34
  0    2019-08-14 15:25:36
  Name: TimeWhen, dtype: object, 5    0.0
  4    0.0
  3    0.0
  2    0.0
  1    0.0
  0    0.0
  Name: IconAltitude, dtype: float64], '08RDE3D00102QZ': [[72    29.150305
   71    29.150334
   70    29.150334
   69    29.150334
   68    29.150047
           ...    
   4     29.150322
   3     29.150322
   2     29.150322
   1     29.150322
   0     29.150322
   Name: Latitude, Length: 73, dtype: float64, 72   -81.048587
   71   -81.048679
   70   -81.048679
   69   -81.048742
   68   -81.048730
           ...    
   4    -81.04807

In [760]:
mapp.mdf.uasFlights[encounterNames[2]][2]

20     76.1
19     75.9
18     75.9
17     76.0
16     76.0
15     75.9
14     75.9
13     75.9
12     75.8
11     75.9
10     76.0
8      76.0
9      76.1
7      76.2
6      76.2
5      76.4
4      80.2
3      76.7
2      99.2
1     109.3
0      83.9
Name: IconAltitude, dtype: float64

In [761]:
mapp.mdf.encounterFlights

['N8480J', 'N20571', 'N1230G', 'N50317', 'N536ND', 'N431RP', 'N714HW']

In [764]:
encounterNames[2]

'07JDE3300201B0'

In [774]:
for p in encounterNames:
    
    pmax = max(mapp.mdf.uasFlights[p][2][:])
    pmin = min(mapp.mdf.uasFlights[p][2][:])
    
    
    
    
    #for i in mapp.mdf.uasFlights[p][2][:]:
    #    print(i)
    
    mapp.mdf.adsbFlightsDictionary['N714HW'+'altitude'][2568]

43.3
43.3
76.4
68.0
109.3
75.8
76.2
75.9
52.0
33.6
59.6
50.0


In [776]:
for i in list(mapp.mdf.uasEncounters[:]):
    print(i)

[['N8480J,428', 'N8480J,429', 'N20571,38', 'N20571,39', 'N1230G,446', 'N1230G,447', 'N1230G,444', 'N1230G,445', 'N8480J,426', 'N8480J,427', 'N20571,40', 'N20571,41', 'N1230G,442', 'N1230G,443', 'N20571,42', 'N20571,43', 'N50317,412', 'N50317,410', 'N50317,411', 'N1230G,441', 'N20571,44', 'N20571,45'], '163DFAS00180PB', '0']
[['N20571,23', 'N50317,430', 'N20571,24', 'N50317,428', 'N50317,429', 'N50317,425', 'N50317,426', 'N50317,427', 'N50317,423', 'N50317,424', 'N1230G,457'], '163DFAS00180PB', '1']
[['N536ND,445', 'N20571,14', 'N431RP,292', 'N431RP,293', 'N431RP,294', 'N431RP,295', 'N431RP,296', 'N431RP,297'], '163DFAS00180PB', '2']
[['N536ND,442', 'N431RP,291', 'N431RP,292', 'N431RP,293', 'N431RP,294', 'N431RP,295'], '163DFAS00180PB', '3']
[['N536ND,441', 'N431RP,290', 'N431RP,291', 'N431RP,292', 'N431RP,293', 'N431RP,294', 'N431RP,295'], '163DFAS00180PB', '4']
[['N20571,8', 'N536ND,439', 'N431RP,289', 'N431RP,290', 'N431RP,291', 'N431RP,292', 'N431RP,293'], '163DFAS00180PB', '5']
[['

In [ ]:



for i in list(mapp.mdf.uasEncounters[:]):
    
    
    for z in i[:][0]:
        
        #pmax = max(mapp.mdf.uasFlights[i][2][:])
        #pmin = min(mapp.mdf.uasFlights[i][2][:])
        
        if (mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'altitude']
        [int(z.split(',')[1])]
        [1]) <pmax:
            print(mapp.mdf.adsbFlightsDictionary[z.split(',')[0]+'altitude']
[int(z.split(',')[1])]
[1])
  


